## SMART project

In [1]:
"""SMART.ipynb

Three level sequential logit model.
Latent variables.
Maximum likelihood (full information) estimation.

:author: Tukey
:date: Wed Jan 20 08:22:28 2021

"""
import pandas as pd
import math
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.distributions as dist
import biogeme.results as res
import biogeme.optimization as opt
import biogeme.messaging as msg
from biogeme.expressions import bioDraws, Beta, DefineVariable, RandomVariable, log, RandomVariable, Integrate, Elem, bioNormalCdf, log, PanelLikelihoodTrajectory, MonteCarlo, exp

In [2]:
# Read the data
df_trip1 = pd.read_csv("exp_data1.csv",sep=',',index_col = 0)

In [3]:
print(df_trip1.shape)
df_trip1 = df_trip1.dropna()
print(df_trip1.shape)

(1926, 133)
(1926, 133)


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
database = db.Database("exp_data1.csv",df_trip1)

#from headers import *
# The following statement allows you to use the names of the variable
# as Python variable.
globals().update(database.variables)

In [6]:
## describe the data
pd.set_option('display.max_columns', None)
df_trip1.describe()

,hhincomemid,hhincomecat,carnum,carnumorg,age,gender,educat,transitpass,drlicense,disability,T1choice,T1WTT,T1BTT,T1TTT,T1RTT,T1ARTT,T1TWT,T1RWT,T1ARWT,T1TTC,T1RTC,T1ARTC,T1CTT,T1CPF,T1ATT,T1APF,T2choice,T2WTT,T2BTT,T2TTT,T2RTT,T2ARTT,T2TWT,T2RWT,T2ARWT,T2TTC,T2RTC,T2ARTC,T2CTT,T2CPF,T2ATT,T2APF,T3choice,T3WTT,T3BTT,T3TTT,T3RTT,T3ARTT,T3TWT,T3RWT,T3ARWT,T3TTC,T3RTC,T3ARTC,T3CTT,T3CPF,T3ATT,T3APF,T4choice,T4WTT,T4BTT,T4TTT,T4RTT,T4ARTT,T4TWT,T4RWT,T4ARWT,T4TTC,T4RTC,T4ARTC,T4CTT,T4CPF,T4ATT,T4APF,I19,I20,I21,I22,I23,I34,I35,I36,I37,I38,NCar,NSDCar,av_transit_trip1,av_transit_trip2,av_transit_trip3,av_transit_trip4,av_rh_trip1,av_rh_trip2,av_rh_trip3,av_rh_trip4,av_arh_trip1,av_arh_trip2,av_arh_trip3,av_arh_trip4,av_car_trip1,av_car_trip2,av_car_trip3,av_car_trip4,av_SDcar_trip1,av_SDcar_trip2,av_SDcar_trip3,av_SDcar_trip4,av_trip1,av_trip2,av_trip3,av_trip4,av_zero_zero,av_zero_one,av_one_zero,av_one_one,PChoice,BuyCar,BuySDCar,SellCar,CMP,AMP,OMP,SMP,MP_zero_zero,MP_zero_one,MP_one_zero,MP_one_one,obs,age_40_less,age_65_more,male,inc_70k_more,edu_college_more,tourcomplexity
count,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.00000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.0,1926.0,1926.000000,1926.000000,1926.000000,1926.000000,1926.0,1926.0,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.0,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000,1926.000000
mean,68317.757009,3.925753,0.959502,1.493769,36.299065,1.531672,3.496885,0.204569,0.989097,1.039460,5.273624,147.186397,61.477155,48.357217,16.409657,16.452233,9.952233,5.421599,5.342679,0.012786,0.130112,0.126425,16.332295,0.012427,16.161994,0.011962,5.256490,181.452233,57.070613,44.862928,15.926791,15.827103,9.875389,4.669782,4.505711,1.659398,17.678609,17.138629,15.792316,1.506334,15.758048,1.453790,85.237799,111.254933,91.170301,90.958463,86.478712,86.405504,85.91433,85.254933,85.228972,84.703531,86.411215,86.733126,86.405504,84.679024,86.441848,84.664486,95.650571,104.985462,97.968847,96.707684,95.948598,95.976636,95.823988,95.684320,95.667186,95.511423,96.196781,96.234683,95.986501,95.508307,95.990135,95.510800,3.672378,3.856698,4.039460,3.736241,3.825545,4.383697,5.050363,4.701973,4.921599,4.835410,0.911215,0.306854,0.722222,0.292316,0.357736,0.364486,0.998962,0.366044,0.362409,0.366044,0.998962,0.365524,0.362409,0.366044,0.631360,0.365005,0.365005,0.365005,0.306854,0.268432,0.268432,0.268432,1.0,1.0,0.146937,0.035826,0.884735,0.884735,1.0,1.0,2.569574,0.331776,0.306854,0.772586,0.133171,0.134116,0.116531,0.008316,0.0,0.134116,0.126839,0.204325,2.517653,0.683801,0.023364,0.529595,0.484943,0.882658,2.182762
std,41265.371997,2.095683,0.779890,0.700807,10.903538,0.501203,2.461072,0.403491,0.103876,0.194737,1

### Structural equations for latent variables

In [7]:
# AV safety
omega_safety = bioDraws('omega_safety','NORMAL')
sigma_safety = Beta('sigma_safety',0,-100,100,0)

coef_intercept_safety = Beta('coef_intercept_safety',1,-100,100,0 )
coef_age_40_less_safety = Beta('coef_age_40_less_safety',0.5,-100,100,0 )
coef_age_65_more_safety = Beta('coef_age_65_more_safety',0,-100,100,0 )
coef_inc_70_more_safety = Beta('coef_inc_70_more_safety',0,-100,100,0 )
coef_male_safety = Beta('coef_male_safety',0,-100,100,0 )
coef_edu_college_safety = Beta('coef_edu_college_safety',0,-100,100,0 )
coef_driver_license_safety = Beta('coef_driver_license_safety',0,-100,100,0 )
coef_transit_pass_safety = Beta('coef_transit_pass_safety',0,-100,100,0 )

Safety = \
            coef_intercept_safety + \
            coef_age_40_less_safety * age_40_less +\
            coef_age_65_more_safety * age_65_more + \
            coef_inc_70_more_safety * inc_70k_more +\
            coef_male_safety * male + \
            coef_edu_college_safety * edu_college_more + \
            coef_driver_license_safety * drlicense + \
            coef_transit_pass_safety * transitpass + \
            sigma_safety * omega_safety

## Car dependency
coef_intercept_cardep = Beta('coef_intercept_cardep',1,-100,100,0 )
coef_age_40_less_cardep = Beta('coef_age_40_less_cardep',0,-100,100,0 )
coef_inc_70_more_cardep = Beta('coef_inc_70_more_cardep',0,-100,100,0 )
coef_male_cardep = Beta('coef_male_cardep',0,-100,100,0 )
coef_age_65_more_cardep = Beta('coef_age_65_more_cardep',0,-100,100,0 )
coef_edu_college_cardep = Beta('coef_edu_college_cardep',0,-100,100,0 )
coef_driver_license_cardep = Beta('coef_driver_license_cardep',0,-100,100,0 )
coef_transit_pass_cardep = Beta('coef_transit_pass_cardep',0,-100,100,0 )
# coef_tourcomplexity_cardep = Beta('coef_tourcomplexity_cardep',0,-100,100,0 )

omega_cardep = bioDraws('omega_cardep','NORMAL')
sigma_cardep = Beta('sigma_cardep',0,-1000,1000,0)
            
Cardep = \
            coef_intercept_cardep +\
            coef_age_40_less_cardep * age_40_less +\
            coef_age_65_more_cardep * age_65_more + \
            coef_inc_70_more_cardep * inc_70k_more +\
            coef_male_cardep * male + \
            coef_edu_college_cardep * edu_college_more + \
            coef_driver_license_cardep * drlicense + \
            coef_transit_pass_cardep * transitpass + \
            sigma_cardep * omega_cardep

# Environmental concerns
# coef_intercept_env = Beta('coef_intercept_env',1,-100,100,0 )
# coef_age_40_less_env = Beta('coef_age_40_less_env',0,-100,100,0 )
# coef_inc_70_more_env = Beta('coef_inc_70_more_env',0,-100,100,0 )
# coef_male_env = Beta('coef_male_env',0,-100,100,0 )
# coef_age_65_more_env = Beta('coef_age_65_more_env',0,-100,100,0 )
# coef_edu_college_env = Beta('coef_edu_college_env',0,-100,100,0 )

# omega_env = bioDraws('omega_env','NORMAL_HALTON3')
# sigma_env = Beta('sigma_env',0,-1000,1000,0)
            
# Env = \
#             coef_intercept_env +\
#             coef_age_40_less_env * age_40_less +\
#             coef_age_65_more_env * age_65_more + \
#             coef_inc_70_more_env * inc_70k_more +\
#             coef_male_env * male + \
#             coef_edu_college_env * edu_college_more + sigma_env * omega_env

# Driving perceptions
# coef_intercept_drvper = Beta('coef_intercept_drvper',1,-100,100,0 )
# coef_age_40_less_drvper = Beta('coef_age_40_less_drvper',0,-100,100,0 )
# coef_inc_70_more_drvper = Beta('coef_inc_70_more_drvper',0,-100,100,0 )
# coef_male_drvper = Beta('coef_male_drvper',0,-100,100,0 )
# coef_age_65_more_drvper = Beta('coef_age_65_more_drvper',0,-100,100,0 )
# coef_edu_college_drvper = Beta('coef_edu_college_drvper',0,-100,100,0 )

# omega_drvper = bioDraws('omega_drvper','NORMAL_HALTON3')
# sigma_drvper = Beta('sigma_drvper',0,-1000,1000,0)
            
# Drvper = \
#             coef_intercept_drvper +\
#             coef_age_40_less_drvper * age_40_less +\
#             coef_age_65_more_drvper * age_65_more + \
#             coef_inc_70_more_drvper * inc_70k_more +\
#             coef_male_drvper * male + \
#             coef_edu_college_drvper * edu_college_more + sigma_drvper * omega_drvper

# ## Attitudes on sharing rides
# coef_intercept_shrds = Beta('coef_intercept_shrds',1,-100,100,0 )
# coef_age_40_less_shrds = Beta('coef_age_40_less_shrds',0,-100,100,0 )
# coef_inc_70_more_shrds = Beta('coef_inc_70_more_shrds',0,-100,100,0 )
# coef_male_shrds = Beta('coef_male_shrds',0,-100,100,0 )
# coef_age_65_more_shrds = Beta('coef_age_65_more_shrds',0,-100,100,0 )
# coef_edu_college_shrds = Beta('coef_edu_college_shrds',0,-100,100,0 )

# omega_shrds = bioDraws('omega_shrds','NORMAL_HALTON3')
# sigma_shrds = Beta('sigma_shrds',0,-1000,1000,0)
            
# Shrds = \
#             coef_intercept_shrds +\
#             coef_age_40_less_shrds * age_40_less +\
#             coef_age_65_more_shrds * age_65_more + \
#             coef_inc_70_more_shrds * inc_70k_more +\
#             coef_male_shrds * male + \
#             coef_edu_college_shrds * edu_college_more + sigma_shrds * omega_shrds

## Ride-sharing perceptions
# coef_intercept_rdper = Beta('coef_intercept_rdper',1,-100,100,0 )
# coef_age_40_less_rdper = Beta('coef_age_40_less_rdper',0,-100,100,0 )
# coef_inc_70_more_rdper = Beta('coef_inc_70_more_rdper',0,-100,100,0 )
# coef_male_rdper = Beta('coef_male_rdper',0,-100,100,0 )
# coef_age_65_more_rdper = Beta('coef_age_65_more_rdper',0,-100,100,0 )
# coef_edu_college_rdper = Beta('coef_edu_college_rdper',0,-100,100,0 )

# omega_rdper = bioDraws('omega_rdper','NORMAL_HALTON3')
# sigma_rdper = Beta('sigma_rdper',0,-1000,1000,0)
            
# Rdper = \
#             coef_intercept_rdper +\
#             coef_age_40_less_rdper * age_40_less +\
#             coef_age_65_more_rdper * age_65_more + \
#             coef_inc_70_more_rdper * inc_70k_more +\
#             coef_male_rdper * male + \
#             coef_edu_college_rdper * edu_college_more + sigma_rdper * omega_rdper

### Measurement equations for latent variables

In [8]:
## AV Safety
B_Safety01_F1 = Beta('B_Safety01_F1',1,None,None,1)
B_Safety02_F1 = Beta('B_Safety02_F1',1,None,None,0)
B_Safety03_F1 = Beta('B_Safety03_F1',1,None,None,0)
B_Safety04_F1 = Beta('B_Safety04_F1',1,None,None,0)
B_Safety05_F1 = Beta('B_Safety05_F1',1,None,None,0)

INTER_Safety01 = Beta('INTER_Safety01',0,None,None,1)
INTER_Safety02 = Beta('INTER_Safety02',0,None,None,1)
INTER_Safety03 = Beta('INTER_Safety03',0,None,None,1)
INTER_Safety04 = Beta('INTER_Safety04',0,None,None,1)
INTER_Safety05 = Beta('INTER_Safety05',0,None,None,1)

MODEL_Safety01 = INTER_Safety01 + B_Safety01_F1 * Safety 
MODEL_Safety02 = INTER_Safety02 + B_Safety02_F1 * Safety 
MODEL_Safety03 = INTER_Safety03 + B_Safety03_F1 * Safety 
MODEL_Safety04 = INTER_Safety04 + B_Safety04_F1 * Safety 
MODEL_Safety05 = INTER_Safety05 + B_Safety05_F1 * Safety

SIGMA_STAR_Safety01 = Beta('SIGMA_STAR_Safety01',1,0,100,1)
SIGMA_STAR_Safety02 = Beta('SIGMA_STAR_Safety02',1,0,100,1)
SIGMA_STAR_Safety03 = Beta('SIGMA_STAR_Safety03',1,0,100,1)
SIGMA_STAR_Safety04 = Beta('SIGMA_STAR_Safety04',1,0,100,1)
SIGMA_STAR_Safety05 = Beta('SIGMA_STAR_Safety05',1,0,100,1)

## Car dependency
B_Cardep01_F1 = Beta('B_Cardep01_F1',1,None,None,1)
B_Cardep02_F1 = Beta('B_Cardep02_F1',1,None,None,0)
B_Cardep03_F1 = Beta('B_Cardep03_F1',1,None,None,0)
B_Cardep04_F1 = Beta('B_Cardep04_F1',1,None,None,0)
B_Cardep05_F1 = Beta('B_Cardep05_F1',1,None,None,0)

INTER_Cardep01 = Beta('INTER_Cardep01',0,None,None,1)
INTER_Cardep02 = Beta('INTER_Cardep02',0.0,None,None,1)
INTER_Cardep03 = Beta('INTER_Cardep03',0.0,None,None,1)
INTER_Cardep04 = Beta('INTER_Cardep04',0.0,None,None,1)
INTER_Cardep05 = Beta('INTER_Cardep05',0.0,None,None,1)

MODEL_Cardep01 = INTER_Cardep01 + B_Cardep01_F1 * Cardep
MODEL_Cardep02 = INTER_Cardep02 + B_Cardep02_F1 * Cardep
MODEL_Cardep03 = INTER_Cardep03 + B_Cardep03_F1 * Cardep
MODEL_Cardep04 = INTER_Cardep04 + B_Cardep04_F1 * Cardep
MODEL_Cardep05 = INTER_Cardep05 + B_Cardep05_F1 * Cardep

SIGMA_STAR_Cardep01 = Beta('SIGMA_STAR_Cardep01',1,0,100,1)
SIGMA_STAR_Cardep02 = Beta('SIGMA_STAR_Cardep02',1,0,100,1)
SIGMA_STAR_Cardep03 = Beta('SIGMA_STAR_Cardep03',1,0,100,1)
SIGMA_STAR_Cardep04 = Beta('SIGMA_STAR_Cardep04',1,0,100,1)
SIGMA_STAR_Cardep05 = Beta('SIGMA_STAR_Cardep05',1,0,100,1)

## Environmental concerns
# B_Env01_F1 = Beta('B_Env01_F1',1,-100,100,1)
# B_Env02_F1 = Beta('B_Env02_F1',0.5,-100,100,0)
# B_Env03_F1 = Beta('B_Env03_F1',0.5,-100,100,0)
# B_Env04_F1 = Beta('B_Env04_F1',0.5,-100,100,0)
# B_Env05_F1 = Beta('B_Env05_F1',0.5,-100,100,0)

# INTER_Env01 = Beta('INTER_Env01',0,-100,100,1)
# INTER_Env02 = Beta('INTER_Env02',0,-100,100,1)
# INTER_Env03 = Beta('INTER_Env03',0,-100,100,1)
# INTER_Env04 = Beta('INTER_Env04',0,-100,100,1)
# INTER_Env05 = Beta('INTER_Env05',0,-100,100,1)

# MODEL_Env01 = INTER_Env01 + B_Env01_F1 * Env
# MODEL_Env02 = INTER_Env02 + B_Env02_F1 * Env
# MODEL_Env03 = INTER_Env03 + B_Env03_F1 * Env
# MODEL_Env04 = INTER_Env04 + B_Env04_F1 * Env
# MODEL_Env05 = INTER_Env05 + B_Env05_F1 * Env

# SIGMA_STAR_Env01 = Beta('SIGMA_STAR_Env01',1,0,100,1)
# SIGMA_STAR_Env02 = Beta('SIGMA_STAR_Env02',1,0,100,1)
# SIGMA_STAR_Env03 = Beta('SIGMA_STAR_Env03',1,0,100,1)
# SIGMA_STAR_Env04 = Beta('SIGMA_STAR_Env04',1,0,100,1)
# SIGMA_STAR_Env05 = Beta('SIGMA_STAR_Env05',1,0,100,1)

# Driving perceptions
# B_Drvper01_F1 = Beta('B_Drvper01_F1',0,-100,100,1)
# B_Drvper02_F1 = Beta('B_Drvper02_F1',0,-100,100,0)
# B_Drvper03_F1 = Beta('B_Drvper03_F1',0,-100,100,0)
# B_Drvper04_F1 = Beta('B_Drvper04_F1',0,-100,100,0)
# B_Drvper05_F1 = Beta('B_Drvper05_F1',0,-100,100,0)

# INTER_Drvper01 = Beta('INTER_Drvper01',0,-100,100,1)
# INTER_Drvper02 = Beta('INTER_Drvper02',0,-100,100,1)
# INTER_Drvper03 = Beta('INTER_Drvper03',0,-100,100,1)
# INTER_Drvper04 = Beta('INTER_Drvper04',0,-100,100,1)
# INTER_Drvper05 = Beta('INTER_Drvper05',0,-100,100,1)

# MODEL_Drvper01 = INTER_Drvper01 + B_Drvper01_F1 * Drvper
# MODEL_Drvper02 = INTER_Drvper02 + B_Drvper02_F1 * Drvper
# MODEL_Drvper03 = INTER_Drvper03 + B_Drvper03_F1 * Drvper
# MODEL_Drvper04 = INTER_Drvper04 + B_Drvper04_F1 * Drvper
# MODEL_Drvper05 = INTER_Drvper05 + B_Drvper05_F1 * Drvper

# SIGMA_STAR_Drvper01 = Beta('SIGMA_STAR_Drvper01',1,0,100,1)
# SIGMA_STAR_Drvper02 = Beta('SIGMA_STAR_Drvper02',1,0,100,1)
# SIGMA_STAR_Drvper03 = Beta('SIGMA_STAR_Drvper03',1,0,100,1)
# SIGMA_STAR_Drvper04 = Beta('SIGMA_STAR_Drvper04',1,0,100,1)
# SIGMA_STAR_Drvper05 = Beta('SIGMA_STAR_Drvper05',1,0,100,1)

# Attitudes on sharing rides
# B_Shrds01_F1 = Beta('B_Shrds01_F1',1,None,None,1)
# B_Shrds02_F1 = Beta('B_Shrds02_F1',1,None,None,0)
# B_Shrds03_F1 = Beta('B_Shrds03_F1',1,None,None,0)
# B_Shrds04_F1 = Beta('B_Shrds04_F1',1,None,None,0)
# B_Shrds05_F1 = Beta('B_Shrds05_F1',1,None,None,0)

# INTER_Shrds01 = Beta('INTER_Shrds01',0,None,None,1)
# INTER_Shrds02 = Beta('INTER_Shrds02',0,None,None,1)
# INTER_Shrds03 = Beta('INTER_Shrds03',0,None,None,1)
# INTER_Shrds04 = Beta('INTER_Shrds04',0,None,None,1)
# INTER_Shrds05 = Beta('INTER_Shrds05',0,None,None,1)

# MODEL_Shrds01 = INTER_Shrds01 + B_Shrds01_F1 * Shrds
# MODEL_Shrds02 = INTER_Shrds02 + B_Shrds02_F1 * Shrds
# MODEL_Shrds03 = INTER_Shrds03 + B_Shrds03_F1 * Shrds
# MODEL_Shrds04 = INTER_Shrds04 + B_Shrds04_F1 * Shrds
# MODEL_Shrds05 = INTER_Shrds05 + B_Shrds05_F1 * Shrds

# SIGMA_STAR_Shrds01 = Beta('SIGMA_STAR_Shrds01',1,0,100,1)
# SIGMA_STAR_Shrds02 = Beta('SIGMA_STAR_Shrds02',1,0,100,1)
# SIGMA_STAR_Shrds03 = Beta('SIGMA_STAR_Shrds03',1,0,100,1)
# SIGMA_STAR_Shrds04 = Beta('SIGMA_STAR_Shrds04',1,0,100,1)
# SIGMA_STAR_Shrds05 = Beta('SIGMA_STAR_Shrds05',1,0,100,1)

# Ride-sharing perceptions
# B_Rdper01_F1 = Beta('B_Rdper01_F1',1,None,None,1)
# B_Rdper02_F1 = Beta('B_Rdper02_F1',1,None,None,0)
# B_Rdper03_F1 = Beta('B_Rdper03_F1',1,None,None,0)
# B_Rdper04_F1 = Beta('B_Rdper04_F1',1,None,None,0)
# B_Rdper05_F1 = Beta('B_Rdper05_F1',1,None,None,0)

# INTER_Rdper01 = Beta('INTER_Rdper01',0,None,None,1)
# INTER_Rdper02 = Beta('INTER_Rdper02',0,None,None,1)
# INTER_Rdper03 = Beta('INTER_Rdper03',0,None,None,1)
# INTER_Rdper04 = Beta('INTER_Rdper04',0,None,None,1)
# INTER_Rdper05 = Beta('INTER_Rdper05',0,None,None,1)

# MODEL_Rdper01 = INTER_Rdper01 + B_Rdper01_F1 * Rdper
# MODEL_Rdper02 = INTER_Rdper02 + B_Rdper02_F1 * Rdper
# MODEL_Rdper03 = INTER_Rdper03 + B_Rdper03_F1 * Rdper
# MODEL_Rdper04 = INTER_Rdper04 + B_Rdper04_F1 * Rdper
# MODEL_Rdper05 = INTER_Rdper05 + B_Rdper05_F1 * Rdper

# SIGMA_STAR_Rdper01 = Beta('SIGMA_STAR_Rdper01',1,0,100,1)
# SIGMA_STAR_Rdper02 = Beta('SIGMA_STAR_Rdper02',1,0,100,1)
# SIGMA_STAR_Rdper03 = Beta('SIGMA_STAR_Rdper03',1,0,100,1)
# SIGMA_STAR_Rdper04 = Beta('SIGMA_STAR_Rdper04',1,0,100,1)
# SIGMA_STAR_Rdper05 = Beta('SIGMA_STAR_Rdper05',1,0,100,1)

## AV safety
delta_1 = Beta('delta_1',1,0,10,0)
delta_2 = Beta('delta_2',1,0,10,0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1 
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

Safety01_tau_1 = (tau_1-MODEL_Safety01) / SIGMA_STAR_Safety01
Safety01_tau_2 = (tau_2-MODEL_Safety01) / SIGMA_STAR_Safety01
Safety01_tau_3 = (tau_3-MODEL_Safety01) / SIGMA_STAR_Safety01
Safety01_tau_4 = (tau_4-MODEL_Safety01) / SIGMA_STAR_Safety01
Safety01_tau_5 = (tau_5-MODEL_Safety01) / SIGMA_STAR_Safety01

Safety02_tau_1 = (tau_1-MODEL_Safety02) / SIGMA_STAR_Safety02
Safety02_tau_2 = (tau_2-MODEL_Safety02) / SIGMA_STAR_Safety02
Safety02_tau_3 = (tau_3-MODEL_Safety02) / SIGMA_STAR_Safety02
Safety02_tau_4 = (tau_4-MODEL_Safety02) / SIGMA_STAR_Safety02
Safety02_tau_5 = (tau_5-MODEL_Safety02) / SIGMA_STAR_Safety02

Safety03_tau_1 = (tau_1-MODEL_Safety03) / SIGMA_STAR_Safety03
Safety03_tau_2 = (tau_2-MODEL_Safety03) / SIGMA_STAR_Safety03
Safety03_tau_3 = (tau_3-MODEL_Safety03) / SIGMA_STAR_Safety03
Safety03_tau_4 = (tau_4-MODEL_Safety03) / SIGMA_STAR_Safety03
Safety03_tau_5 = (tau_5-MODEL_Safety03) / SIGMA_STAR_Safety03

Safety04_tau_1 = (tau_1-MODEL_Safety04) / SIGMA_STAR_Safety04
Safety04_tau_2 = (tau_2-MODEL_Safety04) / SIGMA_STAR_Safety04
Safety04_tau_3 = (tau_3-MODEL_Safety04) / SIGMA_STAR_Safety04
Safety04_tau_4 = (tau_4-MODEL_Safety04) / SIGMA_STAR_Safety04
Safety04_tau_5 = (tau_5-MODEL_Safety04) / SIGMA_STAR_Safety04

Safety05_tau_1 = (tau_1-MODEL_Safety05) / SIGMA_STAR_Safety05
Safety05_tau_2 = (tau_2-MODEL_Safety05) / SIGMA_STAR_Safety05
Safety05_tau_3 = (tau_3-MODEL_Safety05) / SIGMA_STAR_Safety05
Safety05_tau_4 = (tau_4-MODEL_Safety05) / SIGMA_STAR_Safety05
Safety05_tau_5 = (tau_5-MODEL_Safety05) / SIGMA_STAR_Safety05

IndSafety01 = {
    1: bioNormalCdf(Safety01_tau_1),
    2: bioNormalCdf(Safety01_tau_2)-bioNormalCdf(Safety01_tau_1),
    3: bioNormalCdf(Safety01_tau_3)-bioNormalCdf(Safety01_tau_2),
    4: bioNormalCdf(Safety01_tau_4)-bioNormalCdf(Safety01_tau_3),
    5: bioNormalCdf(Safety01_tau_5)-bioNormalCdf(Safety01_tau_4),
    6: 1-bioNormalCdf(Safety01_tau_5),
    9999: 1.0
}

P_Safety01 = Elem(IndSafety01, I19)

IndSafety02 = {
    1: bioNormalCdf(Safety02_tau_1),
    2: bioNormalCdf(Safety02_tau_2)-bioNormalCdf(Safety02_tau_1),
    3: bioNormalCdf(Safety02_tau_3)-bioNormalCdf(Safety02_tau_2),
    4: bioNormalCdf(Safety02_tau_4)-bioNormalCdf(Safety02_tau_3),
    5: bioNormalCdf(Safety02_tau_5)-bioNormalCdf(Safety02_tau_4),
    6: 1-bioNormalCdf(Safety02_tau_5),
    9999: 1.0
}

P_Safety02 = Elem(IndSafety02, I20)

IndSafety03 = {
    1: bioNormalCdf(Safety03_tau_1),
    2: bioNormalCdf(Safety03_tau_2)-bioNormalCdf(Safety03_tau_1),
    3: bioNormalCdf(Safety03_tau_3)-bioNormalCdf(Safety03_tau_2),
    4: bioNormalCdf(Safety03_tau_4)-bioNormalCdf(Safety03_tau_3),
    5: bioNormalCdf(Safety03_tau_5)-bioNormalCdf(Safety03_tau_4),
    6: 1-bioNormalCdf(Safety03_tau_5),
    9999: 1.0
}

P_Safety03 = Elem(IndSafety03, I21)

IndSafety04 = {
    1: bioNormalCdf(Safety04_tau_1),
    2: bioNormalCdf(Safety04_tau_2)-bioNormalCdf(Safety04_tau_1),
    3: bioNormalCdf(Safety04_tau_3)-bioNormalCdf(Safety04_tau_2),
    4: bioNormalCdf(Safety04_tau_4)-bioNormalCdf(Safety04_tau_3),
    5: bioNormalCdf(Safety04_tau_5)-bioNormalCdf(Safety04_tau_4),
    6: 1-bioNormalCdf(Safety04_tau_5),
    9999: 1.0
}

P_Safety04 = Elem(IndSafety04, I22)

IndSafety05 = {
    1: bioNormalCdf(Safety05_tau_1),
    2: bioNormalCdf(Safety05_tau_2)-bioNormalCdf(Safety05_tau_1),
    3: bioNormalCdf(Safety05_tau_3)-bioNormalCdf(Safety05_tau_2),
    4: bioNormalCdf(Safety05_tau_4)-bioNormalCdf(Safety05_tau_3),
    5: bioNormalCdf(Safety05_tau_5)-bioNormalCdf(Safety05_tau_4),
    6: 1-bioNormalCdf(Safety05_tau_5),
    9999: 1.0
}

P_Safety05 = Elem(IndSafety05, I23)


## Car dependency
delta_1_p = Beta('delta_1_p',1,0,10,0 )
delta_2_p = Beta('delta_2_p',1,0,10,0 )
tau_1_p = -delta_1_p - delta_2_p
tau_2_p = -delta_1_p 
tau_3_p = 0
tau_4_p = delta_1_p
tau_5_p = delta_1_p + delta_2_p

Cardep01_tau_1 = (tau_1_p-MODEL_Cardep01) / SIGMA_STAR_Cardep01
Cardep01_tau_2 = (tau_2_p-MODEL_Cardep01) / SIGMA_STAR_Cardep01
Cardep01_tau_3 = (tau_3_p-MODEL_Cardep01) / SIGMA_STAR_Cardep01
Cardep01_tau_4 = (tau_4_p-MODEL_Cardep01) / SIGMA_STAR_Cardep01
Cardep01_tau_5 = (tau_5_p-MODEL_Cardep01) / SIGMA_STAR_Cardep01

Cardep02_tau_1 = (tau_1_p-MODEL_Cardep02) / SIGMA_STAR_Cardep02
Cardep02_tau_2 = (tau_2_p-MODEL_Cardep02) / SIGMA_STAR_Cardep02
Cardep02_tau_3 = (tau_3_p-MODEL_Cardep02) / SIGMA_STAR_Cardep02
Cardep02_tau_4 = (tau_4_p-MODEL_Cardep02) / SIGMA_STAR_Cardep02
Cardep02_tau_5 = (tau_5_p-MODEL_Cardep02) / SIGMA_STAR_Cardep02

Cardep03_tau_1 = (tau_1_p-MODEL_Cardep03) / SIGMA_STAR_Cardep03
Cardep03_tau_2 = (tau_2_p-MODEL_Cardep03) / SIGMA_STAR_Cardep03
Cardep03_tau_3 = (tau_3_p-MODEL_Cardep03) / SIGMA_STAR_Cardep03
Cardep03_tau_4 = (tau_4_p-MODEL_Cardep03) / SIGMA_STAR_Cardep03
Cardep03_tau_5 = (tau_5_p-MODEL_Cardep03) / SIGMA_STAR_Cardep03

Cardep04_tau_1 = (tau_1_p-MODEL_Cardep04) / SIGMA_STAR_Cardep04
Cardep04_tau_2 = (tau_2_p-MODEL_Cardep04) / SIGMA_STAR_Cardep04
Cardep04_tau_3 = (tau_3_p-MODEL_Cardep04) / SIGMA_STAR_Cardep04
Cardep04_tau_4 = (tau_4_p-MODEL_Cardep04) / SIGMA_STAR_Cardep04
Cardep04_tau_5 = (tau_5_p-MODEL_Cardep04) / SIGMA_STAR_Cardep04

Cardep05_tau_1 = (tau_1_p-MODEL_Cardep05) / SIGMA_STAR_Cardep05
Cardep05_tau_2 = (tau_2_p-MODEL_Cardep05) / SIGMA_STAR_Cardep05
Cardep05_tau_3 = (tau_3_p-MODEL_Cardep05) / SIGMA_STAR_Cardep05
Cardep05_tau_4 = (tau_4_p-MODEL_Cardep05) / SIGMA_STAR_Cardep05
Cardep05_tau_5 = (tau_5_p-MODEL_Cardep05) / SIGMA_STAR_Cardep05


IndCardep01 = {
    1: bioNormalCdf(Cardep01_tau_1),
    2: bioNormalCdf(Cardep01_tau_2)-bioNormalCdf(Cardep01_tau_1),
    3: bioNormalCdf(Cardep01_tau_3)-bioNormalCdf(Cardep01_tau_2),
    4: bioNormalCdf(Cardep01_tau_4)-bioNormalCdf(Cardep01_tau_3),
    5: bioNormalCdf(Cardep01_tau_5)-bioNormalCdf(Cardep01_tau_4),
    6: 1-bioNormalCdf(Cardep01_tau_5),
    9999: 1.0
}

P_Cardep01 = Elem(IndCardep01, I34)

IndCardep02 = {
    1: bioNormalCdf(Cardep02_tau_1),
    2: bioNormalCdf(Cardep02_tau_2)-bioNormalCdf(Cardep02_tau_1),
    3: bioNormalCdf(Cardep02_tau_3)-bioNormalCdf(Cardep02_tau_2),
    4: bioNormalCdf(Cardep02_tau_4)-bioNormalCdf(Cardep02_tau_3),
    5: bioNormalCdf(Cardep02_tau_5)-bioNormalCdf(Cardep02_tau_4),
    6: 1-bioNormalCdf(Cardep02_tau_5),
    9999: 1.0
}

P_Cardep02 = Elem(IndCardep02, I35)

IndCardep03 = {
    1: bioNormalCdf(Cardep03_tau_1),
    2: bioNormalCdf(Cardep03_tau_2)-bioNormalCdf(Cardep03_tau_1),
    3: bioNormalCdf(Cardep03_tau_3)-bioNormalCdf(Cardep03_tau_2),
    4: bioNormalCdf(Cardep03_tau_4)-bioNormalCdf(Cardep03_tau_3),
    5: bioNormalCdf(Cardep03_tau_5)-bioNormalCdf(Cardep03_tau_4),
    6: 1-bioNormalCdf(Cardep03_tau_5),
    9999: 1.0
}

P_Cardep03 = Elem(IndCardep03, I36)

IndCardep04 = {
    1: bioNormalCdf(Cardep04_tau_1),
    2: bioNormalCdf(Cardep04_tau_2)-bioNormalCdf(Cardep04_tau_1),
    3: bioNormalCdf(Cardep04_tau_3)-bioNormalCdf(Cardep04_tau_2),
    4: bioNormalCdf(Cardep04_tau_4)-bioNormalCdf(Cardep04_tau_3),
    5: bioNormalCdf(Cardep04_tau_5)-bioNormalCdf(Cardep04_tau_4),
    6: 1-bioNormalCdf(Cardep04_tau_5),
    9999: 1.0
}

P_Cardep04 = Elem(IndCardep04, I37)

IndCardep05 = {
    1: bioNormalCdf(Cardep05_tau_1),
    2: bioNormalCdf(Cardep05_tau_2)-bioNormalCdf(Cardep05_tau_1),
    3: bioNormalCdf(Cardep05_tau_3)-bioNormalCdf(Cardep05_tau_2),
    4: bioNormalCdf(Cardep05_tau_4)-bioNormalCdf(Cardep05_tau_3),
    5: bioNormalCdf(Cardep05_tau_5)-bioNormalCdf(Cardep05_tau_4),
    6: 1-bioNormalCdf(Cardep05_tau_5),
    9999: 1.0
}

P_Cardep05 = Elem(IndCardep05, I38)

## Environmental concerns
# delta_1_e = Beta('delta_1_e',1,0,10,0 )
# delta_2_e = Beta('delta_2_e',1,0,10,0 )
# tau_1_e = -delta_1_e - delta_2_e
# tau_2_e = -delta_1_e 
# tau_3_e = 0
# tau_4_e = delta_1_e
# tau_5_e = delta_1_e + delta_2_e

# Env01_tau_1 = (tau_1_e-MODEL_Env01) / SIGMA_STAR_Env01
# Env01_tau_2 = (tau_2_e-MODEL_Env01) / SIGMA_STAR_Env01
# Env01_tau_3 = (tau_3_e-MODEL_Env01) / SIGMA_STAR_Env01
# Env01_tau_4 = (tau_4_e-MODEL_Env01) / SIGMA_STAR_Env01
# Env01_tau_5 = (tau_5_e-MODEL_Env01) / SIGMA_STAR_Env01

# Env02_tau_1 = (tau_1_e-MODEL_Env02) / SIGMA_STAR_Env02
# Env02_tau_2 = (tau_2_e-MODEL_Env02) / SIGMA_STAR_Env02
# Env02_tau_3 = (tau_3_e-MODEL_Env02) / SIGMA_STAR_Env02
# Env02_tau_4 = (tau_4_e-MODEL_Env02) / SIGMA_STAR_Env02
# Env02_tau_5 = (tau_5_e-MODEL_Env02) / SIGMA_STAR_Env02

# Env03_tau_1 = (tau_1_e-MODEL_Env03) / SIGMA_STAR_Env03
# Env03_tau_2 = (tau_2_e-MODEL_Env03) / SIGMA_STAR_Env03
# Env03_tau_3 = (tau_3_e-MODEL_Env03) / SIGMA_STAR_Env03
# Env03_tau_4 = (tau_4_e-MODEL_Env03) / SIGMA_STAR_Env03
# Env03_tau_5 = (tau_5_p-MODEL_Env03) / SIGMA_STAR_Env03

# Env04_tau_1 = (tau_1_e-MODEL_Env04) / SIGMA_STAR_Env04
# Env04_tau_2 = (tau_2_e-MODEL_Env04) / SIGMA_STAR_Env04
# Env04_tau_3 = (tau_3_e-MODEL_Env04) / SIGMA_STAR_Env04
# Env04_tau_4 = (tau_4_e-MODEL_Env04) / SIGMA_STAR_Env04
# Env04_tau_5 = (tau_5_e-MODEL_Env04) / SIGMA_STAR_Env04

# Env05_tau_1 = (tau_1_e-MODEL_Env05) / SIGMA_STAR_Env05
# Env05_tau_2 = (tau_2_e-MODEL_Env05) / SIGMA_STAR_Env05
# Env05_tau_3 = (tau_3_e-MODEL_Env05) / SIGMA_STAR_Env05
# Env05_tau_4 = (tau_4_e-MODEL_Env05) / SIGMA_STAR_Env05
# Env05_tau_5 = (tau_5_e-MODEL_Env05) / SIGMA_STAR_Env05


# IndEnv01 = {
#     1: bioNormalCdf(Env01_tau_1),
#     2: bioNormalCdf(Env01_tau_2)-bioNormalCdf(Env01_tau_1),
#     3: bioNormalCdf(Env01_tau_3)-bioNormalCdf(Env01_tau_2),
#     4: bioNormalCdf(Env01_tau_4)-bioNormalCdf(Env01_tau_3),
#     5: bioNormalCdf(Env01_tau_5)-bioNormalCdf(Env01_tau_4),
#     6: 1-bioNormalCdf(Env01_tau_5),
#     9999: 1.0
# }

# P_Env01 = Elem(IndEnv01, I51)

# IndEnv02 = {
#     1: bioNormalCdf(Env02_tau_1),
#     2: bioNormalCdf(Env02_tau_2)-bioNormalCdf(Env02_tau_1),
#     3: bioNormalCdf(Env02_tau_3)-bioNormalCdf(Env02_tau_2),
#     4: bioNormalCdf(Env02_tau_4)-bioNormalCdf(Env02_tau_3),
#     5: bioNormalCdf(Env02_tau_5)-bioNormalCdf(Env02_tau_4),
#     6: 1-bioNormalCdf(Env02_tau_5),
#     9999: 1.0
# }

# P_Env02 = Elem(IndEnv02, I52)

# IndEnv03 = {
#     1: bioNormalCdf(Env03_tau_1),
#     2: bioNormalCdf(Env03_tau_2)-bioNormalCdf(Env03_tau_1),
#     3: bioNormalCdf(Env03_tau_3)-bioNormalCdf(Env03_tau_2),
#     4: bioNormalCdf(Env03_tau_4)-bioNormalCdf(Env03_tau_3),
#     5: bioNormalCdf(Env03_tau_5)-bioNormalCdf(Env03_tau_4),
#     6: 1-bioNormalCdf(Env03_tau_5),
#     9999: 1.0
# }

# P_Env03 = Elem(IndEnv03, I53)

# IndEnv04 = {
#     1: bioNormalCdf(Env04_tau_1),
#     2: bioNormalCdf(Env04_tau_2)-bioNormalCdf(Env04_tau_1),
#     3: bioNormalCdf(Env04_tau_3)-bioNormalCdf(Env04_tau_2),
#     4: bioNormalCdf(Env04_tau_4)-bioNormalCdf(Env04_tau_3),
#     5: bioNormalCdf(Env04_tau_5)-bioNormalCdf(Env04_tau_4),
#     6: 1-bioNormalCdf(Env04_tau_5),
#     9999: 1.0
# }

# P_Env04 = Elem(IndEnv04, I60)

# IndEnv05 = {
#     1: bioNormalCdf(Env05_tau_1),
#     2: bioNormalCdf(Env05_tau_2)-bioNormalCdf(Env05_tau_1),
#     3: bioNormalCdf(Env05_tau_3)-bioNormalCdf(Env05_tau_2),
#     4: bioNormalCdf(Env05_tau_4)-bioNormalCdf(Env05_tau_3),
#     5: bioNormalCdf(Env05_tau_5)-bioNormalCdf(Env05_tau_4),
#     6: 1-bioNormalCdf(Env05_tau_5),
#     9999: 1.0
# }

# P_Env05 = Elem(IndEnv05, I61)

## Driving perceptions
# delta_1_d = Beta('delta_1_d',1,0,10,0 )
# delta_2_d = Beta('delta_2_d',1,0,10,0 )
# tau_1_d = -delta_1_d - delta_2_d
# tau_2_d = -delta_1_d 
# tau_3_d = 0
# tau_4_d = delta_1_d
# tau_5_d = delta_1_d + delta_2_d

# Drvper01_tau_1 = (tau_1_d-MODEL_Drvper01) / SIGMA_STAR_Drvper01
# Drvper01_tau_2 = (tau_2_d-MODEL_Drvper01) / SIGMA_STAR_Drvper01
# Drvper01_tau_3 = (tau_3_d-MODEL_Drvper01) / SIGMA_STAR_Drvper01
# Drvper01_tau_4 = (tau_4_d-MODEL_Drvper01) / SIGMA_STAR_Drvper01
# Drvper01_tau_5 = (tau_5_d-MODEL_Drvper01) / SIGMA_STAR_Drvper01

# Drvper02_tau_1 = (tau_1_d-MODEL_Drvper02) / SIGMA_STAR_Drvper02
# Drvper02_tau_2 = (tau_2_d-MODEL_Drvper02) / SIGMA_STAR_Drvper02
# Drvper02_tau_3 = (tau_3_d-MODEL_Drvper02) / SIGMA_STAR_Drvper02
# Drvper02_tau_4 = (tau_4_d-MODEL_Drvper02) / SIGMA_STAR_Drvper02
# Drvper02_tau_5 = (tau_5_d-MODEL_Drvper02) / SIGMA_STAR_Drvper02

# Drvper03_tau_1 = (tau_1_d-MODEL_Drvper03) / SIGMA_STAR_Drvper03
# Drvper03_tau_2 = (tau_2_d-MODEL_Drvper03) / SIGMA_STAR_Drvper03
# Drvper03_tau_3 = (tau_3_d-MODEL_Drvper03) / SIGMA_STAR_Drvper03
# Drvper03_tau_4 = (tau_4_d-MODEL_Drvper03) / SIGMA_STAR_Drvper03
# Drvper03_tau_5 = (tau_5_d-MODEL_Drvper03) / SIGMA_STAR_Drvper03

# Drvper04_tau_1 = (tau_1_d-MODEL_Drvper04) / SIGMA_STAR_Drvper04
# Drvper04_tau_2 = (tau_2_d-MODEL_Drvper04) / SIGMA_STAR_Drvper04
# Drvper04_tau_3 = (tau_3_d-MODEL_Drvper04) / SIGMA_STAR_Drvper04
# Drvper04_tau_4 = (tau_4_d-MODEL_Drvper04) / SIGMA_STAR_Drvper04
# Drvper04_tau_5 = (tau_5_d-MODEL_Drvper04) / SIGMA_STAR_Drvper04

# Drvper05_tau_1 = (tau_1_d-MODEL_Drvper05) / SIGMA_STAR_Drvper05
# Drvper05_tau_2 = (tau_2_d-MODEL_Drvper05) / SIGMA_STAR_Drvper05
# Drvper05_tau_3 = (tau_3_d-MODEL_Drvper05) / SIGMA_STAR_Drvper05
# Drvper05_tau_4 = (tau_4_d-MODEL_Drvper05) / SIGMA_STAR_Drvper05
# Drvper05_tau_5 = (tau_5_d-MODEL_Drvper05) / SIGMA_STAR_Drvper05


# IndDrvper01 = {
#     1: bioNormalCdf(Drvper01_tau_1),
#     2: bioNormalCdf(Drvper01_tau_2)-bioNormalCdf(Drvper01_tau_1),
#     3: bioNormalCdf(Drvper01_tau_3)-bioNormalCdf(Drvper01_tau_2),
#     4: bioNormalCdf(Drvper01_tau_4)-bioNormalCdf(Drvper01_tau_3),
#     5: bioNormalCdf(Drvper01_tau_5)-bioNormalCdf(Drvper01_tau_4),
#     6: 1-bioNormalCdf(Drvper01_tau_5),
#     9999: 1.0
# }

# P_Drvper01 = Elem(IndDrvper01, I54)

# IndDrvper02 = {
#     1: bioNormalCdf(Drvper02_tau_1),
#     2: bioNormalCdf(Drvper02_tau_2)-bioNormalCdf(Drvper02_tau_1),
#     3: bioNormalCdf(Drvper02_tau_3)-bioNormalCdf(Drvper02_tau_2),
#     4: bioNormalCdf(Drvper02_tau_4)-bioNormalCdf(Drvper02_tau_3),
#     5: bioNormalCdf(Drvper02_tau_5)-bioNormalCdf(Drvper02_tau_4),
#     6: 1-bioNormalCdf(Drvper02_tau_5),
#     9999: 1.0
# }

# P_Drvper02 = Elem(IndDrvper02, I56)

# IndDrvper03 = {
#     1: bioNormalCdf(Drvper03_tau_1),
#     2: bioNormalCdf(Drvper03_tau_2)-bioNormalCdf(Drvper03_tau_1),
#     3: bioNormalCdf(Drvper03_tau_3)-bioNormalCdf(Drvper03_tau_2),
#     4: bioNormalCdf(Drvper03_tau_4)-bioNormalCdf(Drvper03_tau_3),
#     5: bioNormalCdf(Drvper03_tau_5)-bioNormalCdf(Drvper03_tau_4),
#     6: 1-bioNormalCdf(Drvper03_tau_5),
#     9999: 1.0
# }

# P_Drvper03 = Elem(IndDrvper03, I57)

# IndDrvper04 = {
#     1: bioNormalCdf(Drvper04_tau_1),
#     2: bioNormalCdf(Drvper04_tau_2)-bioNormalCdf(Drvper04_tau_1),
#     3: bioNormalCdf(Drvper04_tau_3)-bioNormalCdf(Drvper04_tau_2),
#     4: bioNormalCdf(Drvper04_tau_4)-bioNormalCdf(Drvper04_tau_3),
#     5: bioNormalCdf(Drvper04_tau_5)-bioNormalCdf(Drvper04_tau_4),
#     6: 1-bioNormalCdf(Drvper04_tau_5),
#     9999: 1.0
# }

# P_Drvper04 = Elem(IndDrvper04, I58)

# IndDrvper05 = {
#     1: bioNormalCdf(Drvper05_tau_1),
#     2: bioNormalCdf(Drvper05_tau_2)-bioNormalCdf(Drvper05_tau_1),
#     3: bioNormalCdf(Drvper05_tau_3)-bioNormalCdf(Drvper05_tau_2),
#     4: bioNormalCdf(Drvper05_tau_4)-bioNormalCdf(Drvper05_tau_3),
#     5: bioNormalCdf(Drvper05_tau_5)-bioNormalCdf(Drvper05_tau_4),
#     6: 1-bioNormalCdf(Drvper05_tau_5),
#     9999: 1.0
# }

# P_Drvper05 = Elem(IndDrvper05, I59)

## Attitudes on sharing rides
# delta_1_s = Beta('delta_1_s',1,0,10,0 )
# delta_2_s = Beta('delta_2_s',1,0,10,0 )
# tau_1_s = -delta_1_s - delta_2_s
# tau_2_s = -delta_1_s 
# tau_3_s = 0
# tau_4_s = delta_1_s
# tau_5_s = delta_1_s + delta_2_s

# Shrds01_tau_1 = (tau_1_s-MODEL_Shrds01) / SIGMA_STAR_Shrds01
# Shrds01_tau_2 = (tau_2_s-MODEL_Shrds01) / SIGMA_STAR_Shrds01
# Shrds01_tau_3 = (tau_3_s-MODEL_Shrds01) / SIGMA_STAR_Shrds01
# Shrds01_tau_4 = (tau_4_s-MODEL_Shrds01) / SIGMA_STAR_Shrds01
# Shrds01_tau_5 = (tau_5_s-MODEL_Shrds01) / SIGMA_STAR_Shrds01

# Shrds02_tau_1 = (tau_1_s-MODEL_Shrds02) / SIGMA_STAR_Shrds02
# Shrds02_tau_2 = (tau_2_s-MODEL_Shrds02) / SIGMA_STAR_Shrds02
# Shrds02_tau_3 = (tau_3_s-MODEL_Shrds02) / SIGMA_STAR_Shrds02
# Shrds02_tau_4 = (tau_4_s-MODEL_Shrds02) / SIGMA_STAR_Shrds02
# Shrds02_tau_5 = (tau_5_s-MODEL_Shrds02) / SIGMA_STAR_Shrds02

# Shrds03_tau_1 = (tau_1_s-MODEL_Shrds03) / SIGMA_STAR_Shrds03
# Shrds03_tau_2 = (tau_2_s-MODEL_Shrds03) / SIGMA_STAR_Shrds03
# Shrds03_tau_3 = (tau_3_s-MODEL_Shrds03) / SIGMA_STAR_Shrds03
# Shrds03_tau_4 = (tau_4_s-MODEL_Shrds03) / SIGMA_STAR_Shrds03
# Shrds03_tau_5 = (tau_5_s-MODEL_Shrds03) / SIGMA_STAR_Shrds03

# Shrds04_tau_1 = (tau_1_s-MODEL_Shrds04) / SIGMA_STAR_Shrds04
# Shrds04_tau_2 = (tau_2_s-MODEL_Shrds04) / SIGMA_STAR_Shrds04
# Shrds04_tau_3 = (tau_3_s-MODEL_Shrds04) / SIGMA_STAR_Shrds04
# Shrds04_tau_4 = (tau_4_s-MODEL_Shrds04) / SIGMA_STAR_Shrds04
# Shrds04_tau_5 = (tau_5_s-MODEL_Shrds04) / SIGMA_STAR_Shrds04

# Shrds05_tau_1 = (tau_1_s-MODEL_Shrds05) / SIGMA_STAR_Shrds05
# Shrds05_tau_2 = (tau_2_s-MODEL_Shrds05) / SIGMA_STAR_Shrds05
# Shrds05_tau_3 = (tau_3_s-MODEL_Shrds05) / SIGMA_STAR_Shrds05
# Shrds05_tau_4 = (tau_4_s-MODEL_Shrds05) / SIGMA_STAR_Shrds05
# Shrds05_tau_5 = (tau_5_s-MODEL_Shrds05) / SIGMA_STAR_Shrds05


# IndShrds01 = {
#     1: bioNormalCdf(Shrds01_tau_1),
#     2: bioNormalCdf(Shrds01_tau_2)-bioNormalCdf(Shrds01_tau_1),
#     3: bioNormalCdf(Shrds01_tau_3)-bioNormalCdf(Shrds01_tau_2),
#     4: bioNormalCdf(Shrds01_tau_4)-bioNormalCdf(Shrds01_tau_3),
#     5: bioNormalCdf(Shrds01_tau_5)-bioNormalCdf(Shrds01_tau_4),
#     6: 1-bioNormalCdf(Shrds01_tau_5),
#     9999: 1.0
# }

# P_Shrds01 = Elem(IndShrds01, I39)

# IndShrds02 = {
#     1: bioNormalCdf(Shrds02_tau_1),
#     2: bioNormalCdf(Shrds02_tau_2)-bioNormalCdf(Shrds02_tau_1),
#     3: bioNormalCdf(Shrds02_tau_3)-bioNormalCdf(Shrds02_tau_2),
#     4: bioNormalCdf(Shrds02_tau_4)-bioNormalCdf(Shrds02_tau_3),
#     5: bioNormalCdf(Shrds02_tau_5)-bioNormalCdf(Shrds02_tau_4),
#     6: 1-bioNormalCdf(Shrds02_tau_5),
#     9999: 1.0
# }

# P_Shrds02 = Elem(IndShrds02, I40)

# IndShrds03 = {
#     1: bioNormalCdf(Shrds03_tau_1),
#     2: bioNormalCdf(Shrds03_tau_2)-bioNormalCdf(Shrds03_tau_1),
#     3: bioNormalCdf(Shrds03_tau_3)-bioNormalCdf(Shrds03_tau_2),
#     4: bioNormalCdf(Shrds03_tau_4)-bioNormalCdf(Shrds03_tau_3),
#     5: bioNormalCdf(Shrds03_tau_5)-bioNormalCdf(Shrds03_tau_4),
#     6: 1-bioNormalCdf(Shrds03_tau_5),
#     9999: 1.0
# }

# P_Shrds03 = Elem(IndShrds03, I41)

# IndShrds04 = {
#     1: bioNormalCdf(Shrds04_tau_1),
#     2: bioNormalCdf(Shrds04_tau_2)-bioNormalCdf(Shrds04_tau_1),
#     3: bioNormalCdf(Shrds04_tau_3)-bioNormalCdf(Shrds04_tau_2),
#     4: bioNormalCdf(Shrds04_tau_4)-bioNormalCdf(Shrds04_tau_3),
#     5: bioNormalCdf(Shrds04_tau_5)-bioNormalCdf(Shrds04_tau_4),
#     6: 1-bioNormalCdf(Shrds04_tau_5),
#     9999: 1.0
# }

# P_Shrds04 = Elem(IndShrds04, I42)

# IndShrds05 = {
#     1: bioNormalCdf(Shrds05_tau_1),
#     2: bioNormalCdf(Shrds05_tau_2)-bioNormalCdf(Shrds05_tau_1),
#     3: bioNormalCdf(Shrds05_tau_3)-bioNormalCdf(Shrds05_tau_2),
#     4: bioNormalCdf(Shrds05_tau_4)-bioNormalCdf(Shrds05_tau_3),
#     5: bioNormalCdf(Shrds05_tau_5)-bioNormalCdf(Shrds05_tau_4),
#     6: 1-bioNormalCdf(Shrds05_tau_5),
#     9999: 1.0
# }

# P_Shrds05 = Elem(IndShrds05, I43)

## Ride-sharing perceptions
# delta_1_r = Beta('delta_1_r',1,0,10,0 )
# delta_2_r = Beta('delta_2_r',1,0,10,0 )
# tau_1_r = -delta_1_r - delta_2_r
# tau_2_r = -delta_1_r
# tau_3_r = 0
# tau_4_r = delta_1_r
# tau_5_r = delta_1_r + delta_2_r

# Rdper01_tau_1 = (tau_1_r-MODEL_Rdper01) / SIGMA_STAR_Rdper01
# Rdper01_tau_2 = (tau_2_r-MODEL_Rdper01) / SIGMA_STAR_Rdper01
# Rdper01_tau_3 = (tau_3_r-MODEL_Rdper01) / SIGMA_STAR_Rdper01
# Rdper01_tau_4 = (tau_4_r-MODEL_Rdper01) / SIGMA_STAR_Rdper01
# Rdper01_tau_5 = (tau_5_r-MODEL_Rdper01) / SIGMA_STAR_Rdper01

# Rdper02_tau_1 = (tau_1_r-MODEL_Rdper02) / SIGMA_STAR_Rdper02
# Rdper02_tau_2 = (tau_2_r-MODEL_Rdper02) / SIGMA_STAR_Rdper02
# Rdper02_tau_3 = (tau_3_r-MODEL_Rdper02) / SIGMA_STAR_Rdper02
# Rdper02_tau_4 = (tau_4_r-MODEL_Rdper02) / SIGMA_STAR_Rdper02
# Rdper02_tau_5 = (tau_5_r-MODEL_Rdper02) / SIGMA_STAR_Rdper02

# Rdper03_tau_1 = (tau_1_r-MODEL_Rdper03) / SIGMA_STAR_Rdper03
# Rdper03_tau_2 = (tau_2_r-MODEL_Rdper03) / SIGMA_STAR_Rdper03
# Rdper03_tau_3 = (tau_3_r-MODEL_Rdper03) / SIGMA_STAR_Rdper03
# Rdper03_tau_4 = (tau_4_r-MODEL_Rdper03) / SIGMA_STAR_Rdper03
# Rdper03_tau_5 = (tau_5_r-MODEL_Rdper03) / SIGMA_STAR_Rdper03

# Rdper04_tau_1 = (tau_1_r-MODEL_Rdper04) / SIGMA_STAR_Rdper04
# Rdper04_tau_2 = (tau_2_r-MODEL_Rdper04) / SIGMA_STAR_Rdper04
# Rdper04_tau_3 = (tau_3_r-MODEL_Rdper04) / SIGMA_STAR_Rdper04
# Rdper04_tau_4 = (tau_4_r-MODEL_Rdper04) / SIGMA_STAR_Rdper04
# Rdper04_tau_5 = (tau_5_r-MODEL_Rdper04) / SIGMA_STAR_Rdper04

# Rdper05_tau_1 = (tau_1_r-MODEL_Rdper05) / SIGMA_STAR_Rdper05
# Rdper05_tau_2 = (tau_2_r-MODEL_Rdper05) / SIGMA_STAR_Rdper05
# Rdper05_tau_3 = (tau_3_r-MODEL_Rdper05) / SIGMA_STAR_Rdper05
# Rdper05_tau_4 = (tau_4_r-MODEL_Rdper05) / SIGMA_STAR_Rdper05
# Rdper05_tau_5 = (tau_5_r-MODEL_Rdper05) / SIGMA_STAR_Rdper05


# IndRdper01 = {
#     1: bioNormalCdf(Rdper01_tau_1),
#     2: bioNormalCdf(Rdper01_tau_2)-bioNormalCdf(Rdper01_tau_1),
#     3: bioNormalCdf(Rdper01_tau_3)-bioNormalCdf(Rdper01_tau_2),
#     4: bioNormalCdf(Rdper01_tau_4)-bioNormalCdf(Rdper01_tau_3),
#     5: bioNormalCdf(Rdper01_tau_5)-bioNormalCdf(Rdper01_tau_4),
#     6: 1-bioNormalCdf(Rdper01_tau_5),
#     9999: 1.0
# }

# P_Rdper01 = Elem(IndRdper01, I28)

# IndRdper02 = {
#     1: bioNormalCdf(Rdper02_tau_1),
#     2: bioNormalCdf(Rdper02_tau_2)-bioNormalCdf(Rdper02_tau_1),
#     3: bioNormalCdf(Rdper02_tau_3)-bioNormalCdf(Rdper02_tau_2),
#     4: bioNormalCdf(Rdper02_tau_4)-bioNormalCdf(Rdper02_tau_3),
#     5: bioNormalCdf(Rdper02_tau_5)-bioNormalCdf(Rdper02_tau_4),
#     6: 1-bioNormalCdf(Rdper02_tau_5),
#     9999: 1.0
# }

# P_Rdper02 = Elem(IndRdper02, I29)

# IndRdper03 = {
#     1: bioNormalCdf(Rdper03_tau_1),
#     2: bioNormalCdf(Rdper03_tau_2)-bioNormalCdf(Rdper03_tau_1),
#     3: bioNormalCdf(Rdper03_tau_3)-bioNormalCdf(Rdper03_tau_2),
#     4: bioNormalCdf(Rdper03_tau_4)-bioNormalCdf(Rdper03_tau_3),
#     5: bioNormalCdf(Rdper03_tau_5)-bioNormalCdf(Rdper03_tau_4),
#     6: 1-bioNormalCdf(Rdper03_tau_5),
#     9999: 1.0
# }

# P_Rdper03 = Elem(IndRdper03, I30)

# IndRdper04 = {
#     1: bioNormalCdf(Rdper04_tau_1),
#     2: bioNormalCdf(Rdper04_tau_2)-bioNormalCdf(Rdper04_tau_1),
#     3: bioNormalCdf(Rdper04_tau_3)-bioNormalCdf(Rdper04_tau_2),
#     4: bioNormalCdf(Rdper04_tau_4)-bioNormalCdf(Rdper04_tau_3),
#     5: bioNormalCdf(Rdper04_tau_5)-bioNormalCdf(Rdper04_tau_4),
#     6: 1-bioNormalCdf(Rdper04_tau_5),
#     9999: 1.0
# }

# P_Rdper04 = Elem(IndRdper04, I32)

# IndRdper05 = {
#     1: bioNormalCdf(Rdper05_tau_1),
#     2: bioNormalCdf(Rdper05_tau_2)-bioNormalCdf(Rdper05_tau_1),
#     3: bioNormalCdf(Rdper05_tau_3)-bioNormalCdf(Rdper05_tau_2),
#     4: bioNormalCdf(Rdper05_tau_4)-bioNormalCdf(Rdper05_tau_3),
#     5: bioNormalCdf(Rdper05_tau_5)-bioNormalCdf(Rdper05_tau_4),
#     6: 1-bioNormalCdf(Rdper05_tau_5),
#     9999: 1.0
# }

# P_Rdper05 = Elem(IndRdper05, I33)

### Lower level: mode choice: MNL

In [9]:
# # Parameters to be estimated
# ## automated vehicles
AV = Beta('AV', 1, None, None, 0)
AV_S = AV * bioDraws('AV_S', 'NORMAL')

## private vehicles
PV = Beta('PV', 1, None, None, 0)
PV_S = PV * bioDraws('PV_S', 'NORMAL')

## Ride-hailing
RH = Beta('RH', 1, None, None, 0)
RH_S = RH * bioDraws('RH_S', 'NORMAL')

# ## Active travel
# AT = Beta('AT', 1, None, None, 0)
# AT_S = AT * bioDraws('AT_S', 'NORMAL')

# one cost/waiting time
ASC_SDC_1 = Beta('ASC_SDC_1', 0.05, -1000, 1000, 0) ## self-driving cars
ASC_WK_1 = Beta('ASC_WK_1', 0.05, -1000, 1000, 0) ## walking
ASC_BK_1 = Beta('ASC_BK_1', 0.05, -1000, 1000, 0) ## biking
ASC_TS_1 = Beta('ASC_TS_1', 0.05, -1000, 1000, 0) ## transit
ASC_RH_1 = Beta('ASC_RH_1', 0.05, -1000, 1000, 0) ## ridehailing
ASC_SDRH_1 = Beta('ASC_SDRH_1', 0.05, -1000, 1000, 0) ## self-driving ride hailing

B_CAR_TT_1 = Beta('B_CAR_TT_1', 0.05, -1000, 1000, 0) ## Car travel time
B_SDC_TT_1 = Beta('B_SDC_TT_1', 0.05, -1000, 1000, 0) ## self-driving car travel time
B_WK_TT_1 = Beta('B_WK_TT_1', 0.05, -1000, 1000, 0) ## walking time
B_BK_TT_1 = Beta('B_BK_TT_1', 0.05, -1000, 1000, 0) ## biking time
B_TS_TT_1 = Beta('B_TS_TT_1', 0.05, -1000, 1000, 0) ## transit travel time
B_RH_TT_1 = Beta('B_RH_TT_1', 0.05, -1000, 1000, 0) ## ride hailing travel time
B_SDRH_TT_1 = Beta('B_SDRH_TT_1', 0.05, -1000, 1000, 0) ## self-driving ride hailing travel time

B_TS_WT_1 = Beta('B_TS_WT_1', 0.05, -1000, 1000, 0) ## transit waiting time
B_RH_WT_1 = Beta('B_RH_WT_1', 0.05, -1000, 1000, 0) ## ride hailing waiting time
B_SDRH_WT_1 = Beta('B_SDRH_WT_1', 0.05, -1000, 1000, 0) ## self-driving ride hailing waiting time

B_COST_1 = Beta('B_COST_1', 0.05, -1000, 1000, 0) ## transit travel cost
B_PARKING1 = Beta('B_PARKING1', 0.05, -1000, 1000, 0) ## car parking fee

## coefficients of latent variables
B_safety = Beta('B_safety',0.5,None, None,0)
##B_cardep = Beta('B_cardep',0.5,None, None,0)
# B_env = Beta('B_env',0,-2000,200,0)
# B_drvper = Beta('B_drvper',0,-2000,200,0)
B_shrds = Beta('B_shrds',0.5,None, None,0)
B_rdper = Beta('B_rdper',0.5,None, None,0)

## definition of the utility functions
Car_1 = B_CAR_TT_1*T1CTT + B_PARKING1*T1CPF + PV_S
SD_car_1 = ASC_SDC_1 + B_SDC_TT_1*T1ATT + B_PARKING1*T1APF + PV_S + AV_S
Walking_1 = ASC_WK_1 + B_WK_TT_1*T1WTT
Biking_1 = ASC_BK_1 + B_BK_TT_1*T1BTT
Transit_1 = ASC_TS_1 + B_TS_TT_1*T1TTT + B_TS_WT_1*T1TWT + B_COST_1*T1TTC 
Ridehailing_1 = ASC_RH_1 + B_RH_TT_1*T1RTT + B_RH_WT_1*T1RWT + B_COST_1*T1RTC + RH_S
SD_ridehailing_1 = ASC_SDRH_1 + B_SDRH_TT_1*T1ARTT + B_SDRH_WT_1*T1RWT + B_COST_1*T1ARTC + RH_S + AV_S


In [22]:
# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage(2)
logger.setDetailed()

## Associate the utility functions with the numbering of the alternatives
V_mode_1 = {
    1:Walking_1,
    2:Biking_1,
    3:Transit_1,
    4:Ridehailing_1,
    5:SD_ridehailing_1,
    6:Car_1,
    7:SD_car_1
}

## Availability of each alternative, as described in the dataset
av_mode_1 = {
    1:1,
    2:1,
    3:av_transit_trip1, ## 0: no transit options (travel time == 0)
    4:av_rh_trip1,  ## 0: no ride hailing options (travel time == 0)
    5:av_arh_trip1, ## 0: no auto ride hailing options (travel time == 0)
    6:av_car_trip1, ## 0: number of cars == 0 in the scenario
    7:av_SDcar_trip1 ## 0: number of SD cars == 0 in the scenario
}

# Definition of the model
logprob_trip1 = models.logit(V_mode_1, av_mode_1, T1choice)

logprob = log(MonteCarlo(logprob_trip1))

# Create the Biogeme object
biogeme_logprob_trip  = bio.BIOGEME(database,logprob,numberOfDraws=500, numberOfThreads = 100)

# Create the Biogeme object
biogeme_logprob_trip.modelName = 'mode choice'

# Estimate the parameters
results_trip1 = biogeme_logprob_trip.estimate()
# Get the results in a pandas table
MDresult = results_trip1.getEstimatedParameters()
MDresult

[22:46:46] < General >   Remove 112 unused variables from the database as only 20 are used.
[22:46:46] < Detailed >  It is suggested to scale the following variables.
[22:46:46] < Detailed >  Multiply T1ARTT by	0.01 because the largest (abs) value is	118.0
[22:46:46] < Detailed >  Multiply T1WTT by	0.001 because the largest (abs) value is	1777.0
[22:46:46] < Detailed >  Multiply T1ATT by	0.01 because the largest (abs) value is	160.0
[22:46:46] < Detailed >  Multiply T1BTT by	0.001 because the largest (abs) value is	693.0
[22:46:46] < Detailed >  Multiply T1RWT by	0.01 because the largest (abs) value is	34.0
[22:46:46] < Detailed >  Multiply T1TTT by	0.001 because the largest (abs) value is	1357.0
[22:46:46] < Detailed >  Multiply T1RTT by	0.01 because the largest (abs) value is	160.0
[22:46:46] < Detailed >  Multiply T1CTT by	0.01 because the largest (abs) value is	118.0
[22:46:46] < Detailed >  To remove this feature, set the parameter suggestScales to False when creating the BIOGEME 

[22:48:15] < Detailed >  Log likelihood (N = 1926):   -1886.98
[22:48:17] < General >   Log likelihood (N = 1926):   -1886.98 Gradient norm:      7e+02  
[22:48:17] < Detailed >  34 f= 0.9797406 projected rel. grad.= 0.035 delta=  0.12 rho=     1 ++
[22:48:19] < Detailed >  Log likelihood (N = 1926):  -1869.051
[22:48:20] < General >   Log likelihood (N = 1926):  -1869.051 Gradient norm:      8e+02  
[22:48:20] < Detailed >  35 f= 0.9704316 projected rel. grad.= 0.049 delta=  0.25 rho=   1.1 ++
[22:48:22] < Detailed >  Log likelihood (N = 1926):  -1847.557
[22:48:24] < General >   Log likelihood (N = 1926):  -1847.557 Gradient norm:      1e+03  
[22:48:24] < Detailed >  36 f= 0.9592717 projected rel. grad.= 0.099 delta=   0.5 rho=  0.95 ++
[22:48:25] < Detailed >  Log likelihood (N = 1926):  -1830.351
[22:48:27] < General >   Log likelihood (N = 1926):  -1830.351 Gradient norm:      3e+03  
[22:48:27] < Detailed >  37 f= 0.9503381 projected rel. grad.=  0.19 delta=   0.5 rho=  0.46 +
[

[22:50:04] < General >   Log likelihood (N = 1926):  -1670.818 Gradient norm:      1e+02  
[22:50:04] < Detailed >  67 f=  0.867507 projected rel. grad.=0.0077 delta=8.4e+06 rho=   1.5 ++
[22:50:05] < Detailed >  Log likelihood (N = 1926):   -1670.04
[22:50:07] < General >   Log likelihood (N = 1926):   -1670.04 Gradient norm:      1e+02  
[22:50:07] < Detailed >  68 f= 0.8671029 projected rel. grad.=0.0053 delta=1.7e+07 rho=   1.6 ++
[22:50:09] < Detailed >  Log likelihood (N = 1926):  -1668.729
[22:50:10] < General >   Log likelihood (N = 1926):  -1668.729 Gradient norm:      1e+02  
[22:50:10] < Detailed >  69 f= 0.8664221 projected rel. grad.=0.0075 delta=3.4e+07 rho=   1.5 ++
[22:50:12] < Detailed >  Log likelihood (N = 1926):  -1667.206
[22:50:14] < General >   Log likelihood (N = 1926):  -1667.206 Gradient norm:      2e+02  
[22:50:14] < Detailed >  70 f= 0.8656311 projected rel. grad.=0.0097 delta=6.7e+07 rho=   1.5 ++
[22:50:15] < Detailed >  Log likelihood (N = 1926):  -1666.

[22:51:54] < Detailed >  Log likelihood (N = 1926):  -1663.655
[22:51:56] < General >   Log likelihood (N = 1926):  -1663.655 Gradient norm:          8  
[22:51:56] < Detailed >  100 f= 0.8637879 projected rel. grad.=0.00054 delta=7.2e+16 rho=   1.6 ++
[22:51:57] < Detailed >  Log likelihood (N = 1926):  -1663.646
[22:51:59] < General >   Log likelihood (N = 1926):  -1663.646 Gradient norm:      1e+01  
[22:51:59] < Detailed >  101 f= 0.8637829 projected rel. grad.=0.00051 delta=1.4e+17 rho=   1.6 ++
[22:52:01] < Detailed >  Log likelihood (N = 1926):  -1663.621
[22:52:02] < General >   Log likelihood (N = 1926):  -1663.621 Gradient norm:      2e+01  
[22:52:02] < Detailed >  102 f= 0.8637699 projected rel. grad.=0.00076 delta=2.9e+17 rho=   1.6 ++
[22:52:04] < Detailed >  Log likelihood (N = 1926):  -1663.565
[22:52:06] < General >   Log likelihood (N = 1926):  -1663.565 Gradient norm:      3e+01  
[22:52:06] < Detailed >  103 f= 0.8637411 projected rel. grad.=0.0014 delta=5.8e+17 rho

[22:53:47] < Detailed >  Log likelihood (N = 1926):  -1662.505
[22:53:49] < General >   Log likelihood (N = 1926):  -1662.505 Gradient norm:      1e+02  
[22:53:49] < Detailed >  133 f= 0.8631906 projected rel. grad.=0.0075 delta=6.2e+26 rho=   1.5 ++
[22:53:51] < Detailed >  Log likelihood (N = 1926):  -1662.343
[22:53:53] < General >   Log likelihood (N = 1926):  -1662.343 Gradient norm:      8e+01  
[22:53:53] < Detailed >  134 f= 0.8631063 projected rel. grad.= 0.006 delta=1.2e+27 rho=   1.4 ++
[22:53:54] < Detailed >  Log likelihood (N = 1926):  -1662.268
[22:53:56] < General >   Log likelihood (N = 1926):  -1662.268 Gradient norm:      4e+01  
[22:53:56] < Detailed >  135 f= 0.8630674 projected rel. grad.= 0.003 delta=2.5e+27 rho=   1.3 ++
[22:53:58] < Detailed >  Log likelihood (N = 1926):  -1662.251
[22:54:00] < General >   Log likelihood (N = 1926):  -1662.251 Gradient norm:      1e+01  
[22:54:00] < Detailed >  136 f= 0.8630586 projected rel. grad.=0.00097 delta= 5e+27 rho=  

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_BK_1,-2.498478,0.619578,-4.032545,5.517594e-05,1.465824,-1.704487,0.088290
ASC_RH_1,-3.970854,0.886039,-4.481581,7.409209e-06,2.742527,-1.447881,0.147650
ASC_SDC_1,3.203339,0.498292,6.428638,1.287526e-10,0.908961,3.524175,0.000425
ASC_SDRH_1,-3.877265,0.869290,-4.460264,8.185867e-06,2.669953,-1.452184,0.146450
ASC_TS_1,-5.284697,0.863206,-6.122176,9.230601e-10,1.784847,-2.960868,0.003068
ASC_WK_1,-3.178318,0.676604,-4.697453,2.634252e-06,1.898547,-1.674079,0.094115
AV,0.028069,0.311157,0.090208,9.281221e-01,0.138073,0.203290,0.838909
B_BK_TT_1,-0.065531,0.007718,-8.490547,0.000000e+00,0.022660,-2.891991,0.003828
B_CAR_TT_1,-0.030207,0.015836,-1.907503,5.645543e-02,0.016957,-1.781350,0.074855
B_COST_1,-5.181476,1.647653,-3.144762,1.662219e-03,5.404899,-0.958663,0.337729


### Simulation

In [24]:
df_trip1 = pd.read_csv("exp_data1.csv",sep=',',index_col = 0)
database = db.Database("exp_data1.csv",df_trip1)
globals().update(database.variables)

## Associate the utility functions with the numbering of the alternatives
V_mode_1 = {
    1:Walking_1,
    2:Biking_1,
    3:Transit_1,
    4:Ridehailing_1,
    5:SD_ridehailing_1,
    6:Car_1,
    7:SD_car_1
}

## Availability of each alternative, as described in the dataset
av_mode_1 = {
    1:1,
    2:1,
    3:av_transit_trip1, ## 0: no transit options (travel time == 0)
    4:av_rh_trip1,  ## 0: no ride hailing options (travel time == 0)
    5:av_arh_trip1, ## 0: no auto ride hailing options (travel time == 0)
    6:av_car_trip1, ## 0: number of cars == 0 in the scenario
    7:av_SDcar_trip1 ## 0: number of SD cars == 0 in the scenario
}

prob_Walking1 = MonteCarlo(models.logit(V_mode_1, av_mode_1, 1))
prob_Biking1 = MonteCarlo(models.logit(V_mode_1, av_mode_1, 2))
prob_Transit1 = MonteCarlo(models.logit(V_mode_1, av_mode_1, 3))
prob_Ridehailing1 = MonteCarlo(models.logit(V_mode_1, av_mode_1, 4))
prob_SD_ridehailing1 = MonteCarlo(models.logit(V_mode_1, av_mode_1, 5))
prob_Car1 = MonteCarlo(models.logit(V_mode_1, av_mode_1, 6))
prob_SD_car1 = MonteCarlo(models.logit(V_mode_1, av_mode_1, 7))

simulate = {'Walking_1': prob_Walking1,
            'Biking_1': prob_Biking1,
            'Transit_1': prob_Transit1,
            'Ridehailing_1': prob_Ridehailing1,
            'SD_ridehailing_1': prob_SD_ridehailing1,
            'Car_1': prob_Car1,
            'SD_car_1': prob_SD_car1}

results = res.bioResults(pickleFile='mode choice.pickle')

# Create the Biogeme object
biosim = bio.BIOGEME(database, simulate, numberOfDraws=500)
biosim.modelName = "Modechoice_simul"

# Simulate the requested quantities. The output is a Pandas data frame
simresults = biosim.simulate(results.getBetaValues())
simresults.mean()

[22:55:37] < General >   Remove 114 unused variables from the database as only 19 are used.
[22:55:37] < Detailed >  It is suggested to scale the following variables.
[22:55:37] < Detailed >  Multiply T1ARTT by	0.01 because the largest (abs) value is	118.0
[22:55:37] < Detailed >  Multiply T1BTT by	0.001 because the largest (abs) value is	693.0
[22:55:37] < Detailed >  Multiply T1TTT by	0.001 because the largest (abs) value is	1357.0
[22:55:37] < Detailed >  Multiply T1CTT by	0.01 because the largest (abs) value is	118.0
[22:55:37] < Detailed >  Multiply T1WTT by	0.001 because the largest (abs) value is	1777.0
[22:55:37] < Detailed >  Multiply T1ATT by	0.01 because the largest (abs) value is	160.0
[22:55:37] < Detailed >  Multiply T1RTT by	0.01 because the largest (abs) value is	160.0
[22:55:37] < Detailed >  Multiply T1RWT by	0.01 because the largest (abs) value is	34.0
[22:55:37] < Detailed >  To remove this feature, set the parameter suggestScales to False when creating the BIOGEME 

Walking_1           0.026505
Biking_1            0.116254
Transit_1           0.044559
Ridehailing_1       0.084742
SD_ridehailing_1    0.094127
Car_1               0.363554
SD_car_1            0.270259
dtype: float64

In [13]:
print("original data:\n", df_trip1.groupby(['T1choice']).agg({'T1choice': 'count'}))


original data:
           T1choice
T1choice          
1               54
2              218
3               88
4              164
5              182
6              703
7              517


In [31]:
517/1889

0.27368978295394386

### Upper level: car ownership

### Car ownership level: structural and measurement equations

In [12]:
MDresult = res.bioResults(pickleFile='mode choice.pickle').getEstimatedParameters()

AV_S = MDresult.loc["AV", "Value"] * bioDraws('AV_S', 'NORMAL')

# ## private vehicles
PV_S = MDresult.loc["PV", "Value"] * bioDraws('PV_S', 'NORMAL')

# ## Ride-hailing
RH_S = MDresult.loc["RH", "Value"] * bioDraws('RH_S', 'NORMAL')

## calculate mode choice utilities
Car = MDresult.loc["B_CAR_TT_1", "Value"]*T1CTT + MDresult.loc["B_PARKING1", "Value"]*T1CPF + PV_S
SD_car = MDresult.loc["ASC_SDC_1", "Value"] + MDresult.loc["B_SDC_TT_1", "Value"]*T1ATT + MDresult.loc["B_PARKING1", "Value"]*T1APF + AV_S + PV_S
Walking = MDresult.loc["ASC_WK_1", "Value"] + MDresult.loc["B_WK_TT_1", "Value"]*T1WTT
Biking = MDresult.loc["ASC_BK_1", "Value"] + MDresult.loc["B_BK_TT_1", "Value"]*T1BTT
Transit = MDresult.loc["ASC_TS_1", "Value"] + MDresult.loc["B_TS_TT_1", "Value"]*T1TTT + MDresult.loc["B_TS_WT_1", "Value"]*T1TWT + MDresult.loc["B_COST_1", "Value"]*T1TTC
Ridehailing = MDresult.loc["ASC_RH_1", "Value"] + MDresult.loc["B_RH_TT_1", "Value"]*T1RTT + MDresult.loc["B_RH_WT_1", "Value"]*T1RWT + MDresult.loc["B_COST_1", "Value"]*T1RTC + RH_S
SD_ridehailing = MDresult.loc["ASC_SDRH_1", "Value"] + MDresult.loc["B_SDRH_TT_1", "Value"]*T1ARTT + MDresult.loc["B_SDRH_WT_1", "Value"]*T1RWT + MDresult.loc["B_COST_1", "Value"]*T1ARTC + AV_S + RH_S 

## 0 cars, 0 SD cars
## Define scale parameters
lambda1 = Beta('lambda1', 0.5, 0.4, 1, 0) ## scale parameter for nest 1
lambda2 = Beta('lambda2', 0.5, 0.4, 1, 0) ## scale parameter for nest 2
lambda3 = Beta('lambda3', 0.5, 0.4, 1, 0) ## scale parameter for nest 3
lambda4 = Beta('lambda4', 0.5, 0.4, 1, 0) ## scale parameter for nest 4

## coefficients of latent variables
B_cardep01 = Beta('B_cardep01',-0.1,None, None,0)
B_safety01 = Beta('B_safety01',-0.1,None, None,0)

## Define car ownership level intercepts
ASC_zero_one = Beta('ASC_zero_one', -0.01, None, None, 0) 
ASC_one_zero = Beta('ASC_one_zero', -0.01, None, None, 0) 
ASC_one_one = Beta('ASC_one_one', -0.01, None, None, 0) 

## Define coefficient for monthly payment
B_MP = Beta('B_MP', -1, -1000, 1000, 0) 

## Define coefficient for travel utility
B_travel_00 = Beta('B_travel_00',0.1,None, None,0)
B_travel_01 = Beta('B_travel_01',0.1,None, None,0)
B_travel_10 = Beta('B_travel_10',0.1,None, None,0)
B_travel_11 = Beta('B_travel_11',0.1,None, None,0)

## Define utility function
## zero cars, zero driverless cars
IV00 = lambda1*log(\
        ##exp(Car/lambda1)*0+\
        exp(Walking/lambda1)+\
        exp(Biking/lambda1) + \
        exp(Transit/lambda1)*av_transit_trip1+\
        exp(Ridehailing/lambda1)*av_rh_trip1+\
        exp(SD_ridehailing/lambda1)*av_arh_trip1 \
        ##exp(SD_car/lambda1)*0))\
        )

## zero cars, one driverless car
IV01 = lambda2*log(\
        ##exp(Car/lambda2)*0+\
        exp(Walking/lambda2)+\
        exp(Biking/lambda2) + \
        exp(Transit/lambda2)*av_transit_trip1+\
        exp(Ridehailing/lambda2)*av_rh_trip1+\
        exp(SD_ridehailing/lambda2)*av_arh_trip1 +\
        exp(SD_car/lambda2)
        )

## one car, zero driverless car
IV10 = lambda3*log(\
        exp(Car/lambda3)+\
        exp(Walking/lambda3)+\
        exp(Biking/lambda3) + \
        exp(Transit/lambda3)*av_transit_trip1+\
        exp(Ridehailing/lambda3)*av_rh_trip1+\
        exp(SD_ridehailing/lambda3)*av_arh_trip1 \
        ##exp(SD_car/lambda3)*0\
        )

## one car, one driverless car
IV11 = lambda4*log(\
        exp(Car/lambda4)+\
        exp(Walking/lambda4)+\
        exp(Biking/lambda4) + \
        exp(Transit/lambda4)*av_transit_trip1+\
        exp(Ridehailing/lambda4)*av_rh_trip1+\
        exp(SD_ridehailing/lambda4)*av_arh_trip1 +\
        exp(SD_car/lambda4)
        )

## Total utility of each car ownership level
zero_zero = IV00 + B_cardep01 * Cardep
zero_one =  ASC_zero_one + IV01 + B_MP*MP_zero_one + B_safety01 * Safety
one_zero =  ASC_one_zero + IV10 + B_MP*MP_one_zero
one_one =  ASC_one_one + IV11 + B_MP*MP_one_one + B_safety01 * Safety

## Associate the utility functions with the numbering of the alternatives
V_profile = {
        1:zero_zero,
        2:zero_one,
        3:one_zero,
        4:one_one
        }

av_profile = {
        1:av_zero_zero,
        2:av_zero_one,
        3:av_one_zero,
        4:av_one_one
        }

# Definition of the model
logprob_car = models.logit(V_profile, av_profile, PChoice)

condlike = logprob_car  * \
          P_Cardep01 * \
          P_Cardep02 * \
          P_Cardep03 * \
          P_Cardep04 * \
          P_Cardep05 * \
          P_Safety01 * \
          P_Safety02 * \
          P_Safety03 * \
          P_Safety04 * \
          P_Safety05

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage(2)
logger.setDetailed()

# We integrate over the random parameters using Monte-Carlo
logprob_car = log(MonteCarlo(condlike))

# Create the Biogeme object
biogeme_logprob_trip  = bio.BIOGEME(database,logprob_car,numberOfDraws=500
                                    , numberOfThreads = 100)

biogeme_logprob_trip.modelName = 'car_ownership-ml-latent-mnl-latent'

# Estimate the parameters
results_trip = biogeme_logprob_trip.estimate()

# Get the results in a pandas table
pandasResults_trip = results_trip.getEstimatedParameters()

pandasResults_trip

[09:00:27] < General >   Remove 91 unused variables from the database as only 42 are used.
[09:00:27] < Detailed >  It is suggested to scale the following variables.
[09:00:27] < Detailed >  Multiply T1RTT by	0.01 because the largest (abs) value is	160.0
[09:00:27] < Detailed >  Multiply T1ARTT by	0.01 because the largest (abs) value is	118.0
[09:00:27] < Detailed >  Multiply T1WTT by	0.001 because the largest (abs) value is	1777.0
[09:00:27] < Detailed >  Multiply T1BTT by	0.001 because the largest (abs) value is	693.0
[09:00:27] < Detailed >  Multiply T1ATT by	0.01 because the largest (abs) value is	160.0
[09:00:27] < Detailed >  Multiply T1RWT by	0.01 because the largest (abs) value is	34.0
[09:00:27] < Detailed >  Multiply T1CTT by	0.01 because the largest (abs) value is	118.0
[09:00:27] < Detailed >  Multiply T1TTT by	0.001 because the largest (abs) value is	1357.0
[09:00:27] < Detailed >  To remove this feature, set the parameter suggestScales to False when creating the BIOGEME o

[09:37:36] < Detailed >  Log likelihood (N = 1926):   -26711.6
[09:38:22] < General >   Log likelihood (N = 1926):   -26711.6 Gradient norm:      2e+02  
[09:38:22] < Detailed >  30 f=  13.86895 projected rel. grad.=0.0017 delta=5.2e+05 rho=   1.4 ++
[09:38:57] < Detailed >  Log likelihood (N = 1926):  -26708.55
[09:39:43] < General >   Log likelihood (N = 1926):  -26708.55 Gradient norm:      1e+02  
[09:39:43] < Detailed >  31 f=  13.86737 projected rel. grad.=0.0014 delta= 1e+06 rho=   1.4 ++
[09:40:19] < Detailed >  Log likelihood (N = 1926):  -26707.27
[09:41:04] < General >   Log likelihood (N = 1926):  -26707.27 Gradient norm:      1e+02  
[09:41:04] < Detailed >  32 f=   13.8667 projected rel. grad.=0.0013 delta=2.1e+06 rho=   1.6 ++
[09:41:40] < Detailed >  Log likelihood (N = 1926):  -26698.68
[09:42:25] < General >   Log likelihood (N = 1926):  -26698.68 Gradient norm:      9e+01  
[09:42:25] < Detailed >  33 f=  13.86225 projected rel. grad.=0.0013 delta=4.2e+06 rho=   1.2 

[10:24:38] < Detailed >  Log likelihood (N = 1926):   -26667.4
[10:25:24] < General >   Log likelihood (N = 1926):   -26667.4 Gradient norm:          7  
[10:25:24] < Detailed >  64 f=    13.846 projected rel. grad.=0.00018 delta= 9e+15 rho=   2.4 ++
[10:26:01] < Detailed >  Log likelihood (N = 1926):  -26667.36
[10:26:48] < General >   Log likelihood (N = 1926):  -26667.36 Gradient norm:          6  
[10:26:48] < Detailed >  65 f=  13.84598 projected rel. grad.=0.00018 delta=1.8e+16 rho=   1.4 ++
[10:27:24] < Detailed >  Log likelihood (N = 1926):  -26667.33
[10:28:11] < General >   Log likelihood (N = 1926):  -26667.33 Gradient norm:          7  
[10:28:11] < Detailed >  66 f=  13.84597 projected rel. grad.=0.00018 delta=3.6e+16 rho=   1.3 ++
[10:28:48] < Detailed >  Log likelihood (N = 1926):  -26667.29
[10:29:34] < General >   Log likelihood (N = 1926):  -26667.29 Gradient norm:          8  
[10:29:34] < Detailed >  67 f=  13.84595 projected rel. grad.=0.00018 delta=7.2e+16 rho=   

[11:09:54] < Detailed >  Log likelihood (N = 1926):   -26666.6
[11:10:41] < General >   Log likelihood (N = 1926):   -26666.6 Gradient norm:          6  
[11:10:41] < Detailed >  97 f=  13.84558 projected rel. grad.=0.00029 delta=  0.62 rho=   1.4 ++
[11:11:18] < Detailed >  Log likelihood (N = 1926):  -26666.56
[11:12:05] < General >   Log likelihood (N = 1926):  -26666.56 Gradient norm:          4  
[11:12:05] < Detailed >  98 f=  13.84557 projected rel. grad.=0.00024 delta=   1.2 rho=   1.4 ++
[11:12:42] < Detailed >  Log likelihood (N = 1926):  -26666.55
[11:13:29] < General >   Log likelihood (N = 1926):  -26666.55 Gradient norm:          3  
[11:13:29] < Detailed >  99 f=  13.84556 projected rel. grad.=0.00016 delta=   2.5 rho=   1.7 ++
[11:14:08] < Detailed >  Log likelihood (N = 1926):  -26666.54
[11:14:56] < General >   Log likelihood (N = 1926):  -26666.54 Gradient norm:          4  
[11:14:56] < Detailed >  100 f=  13.84555 projected rel. grad.=8.8e-05 delta=   4.9 rho=   1.

[11:52:12] < Detailed >  Log likelihood (N = 1926):  -26666.27
[11:53:00] < General >   Log likelihood (N = 1926):  -26666.27 Gradient norm:          2  
[11:53:00] < Detailed >  137 f=  13.84541 projected rel. grad.= 4e-05 delta=3.8e-05 rho=   1.2 ++
[11:53:37] < Detailed >  Log likelihood (N = 1926):   -26666.3
[11:53:37] < Detailed >  138 f=  13.84541 projected rel. grad.= 4e-05 delta=1.9e-05 rho=-1.2e+02 -
[11:54:14] < Detailed >  Log likelihood (N = 1926):   -26666.3
[11:54:14] < Detailed >  139 f=  13.84541 projected rel. grad.= 4e-05 delta=9.6e-06 rho=-2.3e+02 -
[11:54:52] < Detailed >  Log likelihood (N = 1926):  -26666.27
[11:55:39] < General >   Log likelihood (N = 1926):  -26666.27 Gradient norm:          2  
[11:55:39] < Detailed >  140 f=  13.84541 projected rel. grad.=3.7e-05 delta=1.9e-05 rho=   1.3 ++
[11:56:16] < Detailed >  Log likelihood (N = 1926):   -26666.3
[11:56:16] < Detailed >  141 f=  13.84541 projected rel. grad.=3.7e-05 delta=9.6e-06 rho=-2.5e+02 -
[11:56:5

[12:30:04] < Detailed >  Log likelihood (N = 1926):   -26666.3
[12:30:04] < Detailed >  183 f=  13.84541 projected rel. grad.=3.6e-05 delta=2.3e-12 rho=-1e+09 -
[12:30:41] < Detailed >  Log likelihood (N = 1926):   -26666.3
[12:30:41] < Detailed >  184 f=  13.84541 projected rel. grad.=3.6e-05 delta=1.1e-12 rho=-2e+09 -
[12:31:17] < Detailed >  Log likelihood (N = 1926):  -26666.27
[12:32:04] < General >   Log likelihood (N = 1926):  -26666.27 Gradient norm:          2  
[12:32:04] < Detailed >  185 f=  13.84541 projected rel. grad.=3.6e-05 delta=1.1e-12 rho=  0.73 +
[12:32:40] < Detailed >  Log likelihood (N = 1926):   -26666.3
[12:32:40] < Detailed >  186 f=  13.84541 projected rel. grad.=3.6e-05 delta=5.7e-13 rho=-4e+09 -
[12:33:17] < Detailed >  Log likelihood (N = 1926):   -26666.3
[12:33:17] < Detailed >  187 f=  13.84541 projected rel. grad.=3.6e-05 delta=2.9e-13 rho=-8e+09 -
[12:33:53] < Detailed >  Log likelihood (N = 1926):  -26666.27
[12:34:40] < General >   Log likelihood (

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_one_one,-5.484605,5.061688e-01,-1.083553e+01,0.000000e+00,0.641541,-8.549107,0.000000e+00
ASC_one_zero,-1.180294,5.275118e-01,-2.237475e+00,2.525531e-02,0.601367,-1.962685,4.968276e-02
ASC_zero_one,-5.356643,5.301698e-01,-1.010364e+01,0.000000e+00,0.594088,-9.016576,0.000000e+00
B_Cardep02_F1,1.803438,4.334045e-02,4.161097e+01,0.000000e+00,0.044167,40.832544,0.000000e+00
B_Cardep03_F1,1.323229,3.286934e-02,4.025725e+01,0.000000e+00,0.033789,39.161046,0.000000e+00
B_Cardep04_F1,1.646719,4.034483e-02,4.081611e+01,0.000000e+00,0.045965,35.825648,0.000000e+00
B_Cardep05_F1,1.444488,3.546905e-02,4.072530e+01,0.000000e+00,0.038403,37.613515,0.000000e+00
B_MP,-5.973283,6.574663e-01,-9.085307e+00,0.000000e+00,1.719053,-3.474752,5.113251e-04
B_Safety02_F1,1.037868,2.378947e-02,4.362718e+01,0.000000e+00,0.021999,47.177138,0.000000e+00
B_Safety03_F1,1.074529,2.480208e-02,4.332416e+01,0.000000e+00,0.026205,41.004121,0.000000e+00


### Simulation

In [78]:
df_trip1 = pd.read_csv("Scenario3.csv",sep=',',index_col = 0)
database = db.Database("Scenario3.csv",df_trip1)
globals().update(database.variables)

MDresult = res.bioResults(pickleFile='mode choice.pickle').getEstimatedParameters()

AV_S = MDresult.loc["AV", "Value"] * bioDraws('AV_S', 'NORMAL')
PV_S = MDresult.loc["PV", "Value"] * bioDraws('PV_S', 'NORMAL')
RH_S = MDresult.loc["RH", "Value"] * bioDraws('RH_S', 'NORMAL')

## calculate mode choice utilities
Car = MDresult.loc["B_CAR_TT_1", "Value"]*T1CTT + MDresult.loc["B_PARKING1", "Value"]*T1CPF + PV_S
SD_car = MDresult.loc["ASC_SDC_1", "Value"] + MDresult.loc["B_SDC_TT_1", "Value"]*T1ATT + MDresult.loc["B_PARKING1", "Value"]*T1APF + AV_S + PV_S
Walking = MDresult.loc["ASC_WK_1", "Value"] + MDresult.loc["B_WK_TT_1", "Value"]*T1WTT
Biking = MDresult.loc["ASC_BK_1", "Value"] + MDresult.loc["B_BK_TT_1", "Value"]*T1BTT
Transit = MDresult.loc["ASC_TS_1", "Value"] + MDresult.loc["B_TS_TT_1", "Value"]*T1TTT + MDresult.loc["B_TS_WT_1", "Value"]*T1TWT + MDresult.loc["B_COST_1", "Value"]*T1TTC
Ridehailing = MDresult.loc["ASC_RH_1", "Value"] + MDresult.loc["B_RH_TT_1", "Value"]*T1RTT + MDresult.loc["B_RH_WT_1", "Value"]*T1RWT + MDresult.loc["B_COST_1", "Value"]*T1RTC + RH_S
SD_ridehailing = MDresult.loc["ASC_SDRH_1", "Value"] + MDresult.loc["B_SDRH_TT_1", "Value"]*T1ARTT + MDresult.loc["B_SDRH_WT_1", "Value"]*T1RWT + MDresult.loc["B_COST_1", "Value"]*T1ARTC+ RH_S + AV_S 

## Define scale parameters
lambda1 = Beta('lambda1', 0.5, 0.4, 1, 0) ## scale parameter for nest 1
lambda2 = Beta('lambda2', 0.5, 0.4, 1, 0) ## scale parameter for nest 2
lambda3 = Beta('lambda3', 0.5, 0.4, 1, 0) ## scale parameter for nest 3
lambda4 = Beta('lambda4', 0.5, 0.4, 1, 0) ## scale parameter for nest 4

## coefficients of latent variables
B_cardep01 = Beta('B_cardep01',-0.1,None, None,0)
B_safety01 = Beta('B_safety01',-0.1,None, None,0)

## Define car ownership level intercepts
ASC_zero_one = Beta('ASC_zero_one', -0.01, None, None, 0) ## biking
ASC_one_zero = Beta('ASC_one_zero', -0.01, None, None, 0) ## transit
ASC_one_one = Beta('ASC_one_one', -0.01, None, None, 0) ## ridehailing

## Define coefficient for monthly payment
B_MP = Beta('B_MP', -1, -1000, 1000, 0) 

IV00 = lambda1*log(\
        ##exp(Car/lambda1)*0+\
        exp(Walking/lambda1)+\
        exp(Biking/lambda1) + \
        exp(Transit/lambda1)*av_transit_trip1+\
        exp(Ridehailing/lambda1)*av_rh_trip1+\
        exp(SD_ridehailing/lambda1)*av_arh_trip1 \
        ##exp(SD_car/lambda1)*0))\
        )

## zero cars, one driverless car
IV01 = lambda2*log(\
        ##exp(Car/lambda2)*0+\
        exp(Walking/lambda2)+\
        exp(Biking/lambda2) + \
        exp(Transit/lambda2)*av_transit_trip1+\
        exp(Ridehailing/lambda2)*av_rh_trip1+\
        exp(SD_ridehailing/lambda2)*av_arh_trip1 +\
        exp(SD_car/lambda2)
        )

## one car, zero driverless car
IV10 = lambda3*log(\
        exp(Car/lambda3)+\
        exp(Walking/lambda3)+\
        exp(Biking/lambda3) + \
        exp(Transit/lambda3)*av_transit_trip1+\
        exp(Ridehailing/lambda3)*av_rh_trip1+\
        exp(SD_ridehailing/lambda3)*av_arh_trip1 \
        ##exp(SD_car/lambda3)*0\
        )

## one car, one driverless car
IV11 = lambda4*log(\
        exp(Car/lambda4)+\
        exp(Walking/lambda4)+\
        exp(Biking/lambda4) + \
        exp(Transit/lambda4)*av_transit_trip1+\
        exp(Ridehailing/lambda4)*av_rh_trip1+\
        exp(SD_ridehailing/lambda4)*av_arh_trip1 +\
        exp(SD_car/lambda4)
        )

## Total utility of each car ownership level
zero_zero = IV00 + B_cardep01 * Cardep
zero_one =  ASC_zero_one + IV01 + B_MP*MP_zero_one + B_safety01 * Safety
one_zero =  ASC_one_zero + IV10 + B_MP*MP_one_zero 
one_one =  ASC_one_one + IV11 + B_MP*MP_one_one + B_safety01 * Safety

V_profile = {
        1:zero_zero,
        2:zero_one,
        3:one_zero,
        4:one_one
        }

av_profile = {
        1:av_zero_zero,
        2:av_zero_one,
        3:av_one_zero,
        4:av_one_one
        }

# Definition of the model
prob_00 = MonteCarlo(models.logit(V_profile, av_profile, 1))
prob_01 = MonteCarlo(models.logit(V_profile, av_profile, 2))
prob_10 = MonteCarlo(models.logit(V_profile, av_profile, 3))
prob_11 = MonteCarlo(models.logit(V_profile, av_profile, 4))

simulate = {'prob_00': prob_00,
            'prob_01': prob_01,
            'prob_10': prob_10,
            'prob_11': prob_11}

results = res.bioResults(pickleFile='car_ownership-ml-latent-mnl-latent.pickle')

# Create the Biogeme object
biosim = bio.BIOGEME(database, simulate, numberOfDraws=500)
biosim.modelName = "Carownership_simul"

# Simulate the requested quantities. The output is a Pandas data frame
simresults = biosim.simulate(results.getBetaValues())
simresults.mean()

[16:38:16] < General >   Remove 103 unused variables from the database as only 31 are used.
[16:38:16] < Detailed >  It is suggested to scale the following variables.
[16:38:16] < Detailed >  Multiply T1RTT by	0.01 because the largest (abs) value is	144.0
[16:38:16] < Detailed >  Multiply T1ARTT by	0.01 because the largest (abs) value is	106.2
[16:38:16] < Detailed >  Multiply T1WTT by	0.001 because the largest (abs) value is	1777.0
[16:38:16] < Detailed >  Multiply T1BTT by	0.001 because the largest (abs) value is	693.0
[16:38:16] < Detailed >  Multiply T1ATT by	0.01 because the largest (abs) value is	160.0
[16:38:16] < Detailed >  Multiply T1RWT by	0.01 because the largest (abs) value is	34.0
[16:38:16] < Detailed >  Multiply T1CTT by	0.01 because the largest (abs) value is	118.0
[16:38:16] < Detailed >  Multiply T1TTT by	0.001 because the largest (abs) value is	1221.3
[16:38:16] < Detailed >  To remove this feature, set the parameter suggestScales to False when creating the BIOGEME 

prob_00    0.203784
prob_01    0.184082
prob_10    0.478651
prob_11    0.133482
dtype: float64

In [15]:
print("original data:\n", df_trip1.groupby(['PChoice']).agg({'PChoice': 'count'}))


original data:
          PChoice
PChoice         
1            377
2            333
3            958
4            258


In [125]:
377/1926

0.19574247144340604

In [119]:
333/1926

0.17289719626168223

In [120]:
958/1926

0.4974039460020768

In [121]:
258/1926

0.13395638629283488